In [1]:
!pip install mediapipe opencv-python

  Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl.metadata (20 kB)
  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl (37.3 MB)
Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [opencv-python]0m [opencv-python]


In [2]:
import mediapipe as mp
import cv2
import numpy as np
import math

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_pushup(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    
    # Variables to track pushup state
    pushup_count = 0
    pushup_stage = None  # "up" or "down"
    
    # Lists to store angles for analysis
    elbow_angles = []
    body_alignment_scores = []
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            # Get key points for pushup analysis
            # Shoulders, elbows, wrists
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate elbow angle (important for pushup depth)
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Get right elbow angle (shoulder-elbow-wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            elbow_angles.append(right_elbow_angle)
            
            # Check body alignment (straight back)
            # Get hip, shoulder, and ankle points
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # Calculate body alignment score (how straight the body is)
            # Higher score means better alignment
            hip_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            hip_ankle_angle = calculate_angle(right_shoulder, right_hip, right_ankle)
            
            # Ideally both angles should be close to 180 degrees in a proper pushup
            alignment_score = (180 - abs(hip_shoulder_angle - 180) - abs(hip_ankle_angle - 180))/180
            body_alignment_scores.append(alignment_score)
            
            # Determine pushup stage based on elbow angle
            if right_elbow_angle > 150:
                pushup_stage = "up"
            
            # If we were in "up" position and now angle is < 90, we're in "down" position
            elif right_elbow_angle < 90 and pushup_stage == "up":
                pushup_stage = "down"
                pushup_count += 1
                
    cap.release()
    
    # Analyze the data collected
    avg_elbow_angle_bottom = min(elbow_angles)
    avg_elbow_angle_top = max(elbow_angles)
    avg_alignment = sum(body_alignment_scores) / len(body_alignment_scores)
    
    # Generate feedback
    feedback = {
        "pushup_count": pushup_count,
        "form_analysis": {
            "elbow_angle_at_bottom": avg_elbow_angle_bottom,
            "elbow_angle_at_top": avg_elbow_angle_top,
            "body_alignment_score": avg_alignment * 100  # Convert to percentage
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if avg_elbow_angle_bottom > 90:
        feedback["feedback"].append("You're not going deep enough. Try to lower your body until your elbows are at 90 degrees.")
    
    if avg_elbow_angle_top < 150:
        feedback["feedback"].append("You're not fully extending your arms at the top of the pushup.")
    
    if avg_alignment < 0.8:
        feedback["feedback"].append("Keep your body straighter throughout the movement. Your hips are sagging or piking.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your pushups have good depth and body alignment.")
        
    return feedback

# Example usage
result = analyze_pushup("pushup.MOV")
print(f"Counted {result['pushup_count']} pushups")
print(f"Body alignment score: {result['form_analysis']['body_alignment_score']:.1f}%")
print("\nFeedback:")
for item in result["feedback"]:
    print(f"- {item}")

I0000 00:00:1745947321.250408 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
OpenCV: Couldn't read video stream from file "pushup.MOV"
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1745947321.329038 6322255 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745947321.343162 6322261 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


ValueError: min() arg is an empty sequence

In [5]:
import mediapipe as mp
import cv2
import numpy as np
import math

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_pushup(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track pushup state
    pushup_count = 0
    pushup_stage = None  # "up" or "down"
    frames_without_detection = 0
    good_frames = 0
    
    # Lists to store angles for analysis
    elbow_angles = []
    body_alignment_scores = []
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            frames_without_detection = 0
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get key points for pushup analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Get right elbow angle (shoulder-elbow-wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            elbow_angles.append(right_elbow_angle)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize elbow angle
            r_shoulder_px = normalized_to_pixel_coordinates(right_shoulder[0], right_shoulder[1], frame_width, frame_height)
            r_elbow_px = normalized_to_pixel_coordinates(right_elbow[0], right_elbow[1], frame_width, frame_height)
            r_wrist_px = normalized_to_pixel_coordinates(right_wrist[0], right_wrist[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"Elbow: {right_elbow_angle:.1f}°",
                        (r_elbow_px[0] - 50, r_elbow_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Body alignment check
            # Get hip, shoulder, and ankle points
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # Calculate body alignment
            hip_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            hip_ankle_angle = calculate_angle(right_shoulder, right_hip, right_ankle)
            
            # Better alignment calculation
            # Perfect alignment would have both angles close to 180
            alignment_score = (180 - abs(hip_shoulder_angle - 180) - abs(hip_ankle_angle - 180))/180
            body_alignment_scores.append(alignment_score)
            
            # Visualize body alignment
            cv2.putText(annotated_image, 
                        f"Alignment: {alignment_score*100:.1f}%",
                        (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
            
            # Determine pushup stage based on elbow angle
            if right_elbow_angle > 150 and pushup_stage == "down":
                pushup_stage = "up"
                # Draw pushup count
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # If we were in "up" position and now angle is < 90, we're in "down" position
            elif right_elbow_angle < 100 and (pushup_stage == "up" or pushup_stage is None):
                pushup_stage = "down"
                pushup_count += 1
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                print(f"Pushup #{pushup_count} detected at frame with elbow angle {right_elbow_angle:.1f}")
        else:
            frames_without_detection += 1
            if frames_without_detection % 30 == 0:
                print(f"No pose detection for {frames_without_detection} frames")
                
        # Display pushup count
        cv2.putText(annotated_image, f'Pushups: {pushup_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path:
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "pushup_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Analyze the data collected
    if elbow_angles:
        avg_elbow_angle_bottom = min(elbow_angles)
        avg_elbow_angle_top = max(elbow_angles)
    else:
        avg_elbow_angle_bottom = 0
        avg_elbow_angle_top = 0
        
    avg_alignment = sum(body_alignment_scores) / max(len(body_alignment_scores), 1)
    
    # Generate feedback
    feedback = {
        "pushup_count": pushup_count,
        "form_analysis": {
            "elbow_angle_at_bottom": avg_elbow_angle_bottom,
            "elbow_angle_at_top": avg_elbow_angle_top,
            "body_alignment_score": avg_alignment * 100,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if avg_elbow_angle_bottom > 100:
        feedback["feedback"].append("You're not going deep enough. Try to lower your body until your elbows are at 90 degrees.")
    
    if avg_elbow_angle_top < 150:
        feedback["feedback"].append("You're not fully extending your arms at the top of the pushup.")
    
    if avg_alignment < 0.8:
        feedback["feedback"].append("Keep your body straighter throughout the movement. Your hips are sagging or piking.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your pushups have good depth and body alignment.")
        
    return feedback

# Example usage
result = analyze_pushup("pushup.MOV", "analyzed_pushup.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('pushup_count', 0)} pushups")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Body alignment score: {result['form_analysis']['body_alignment_score']:.1f}%")
    print(f"Lowest elbow angle: {result['form_analysis']['elbow_angle_at_bottom']:.1f}°")
    print(f"Highest elbow angle: {result['form_analysis']['elbow_angle_at_top']:.1f}°")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1743722914.783497 14958205 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4
W0000 00:00:1743722914.855040 14971231 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743722914.866219 14971231 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29
Detected 33 landmarks
Pushup #1 detected at frame with elbow angle 97.1

=== ANALYSIS RESULTS ===
Counted 1 pushups
Body alignment score: 10.5%
Lowest elbow angle: 6.9°
Highest elbow angle: 126.2°
Frames analyzed: 86

Feedback:
- You're not fully extending your arms at the top of the pushup.
- Keep your body straighter throughout the movement. Your hips are sagging or piking.


In [9]:
import mediapipe as mp
import cv2
import numpy as np
import math

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_bicep_curl(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track bicep curl state
    curl_count = 0
    curl_stage = None  # "up" or "down"
    good_frames = 0
    
    # Lists to store angles and positions for analysis
    elbow_angles = []
    shoulder_stability = []
    hip_stability = []
    wrist_angles = []
    rep_depths = []  # To track curl depth
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get key points for curl analysis
            # We'll focus on right arm for simplicity, but you could analyze both
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Get elbow angle (shoulder-elbow-wrist)
            elbow_angle = calculate_angle(shoulder, elbow, wrist)
            elbow_angles.append(elbow_angle)
            
            # Track shoulder stability - shoulder should remain still relative to hip
            shoulder_hip_distance = np.sqrt((shoulder[0] - hip[0])**2 + (shoulder[1] - hip[1])**2)
            shoulder_stability.append(shoulder_hip_distance)
            
            # Measure wrist angle relative to forearm (to detect wrist curling/improper form)
            # For simplicity, we'll estimate this as deviation from straight line
            wrist_deviation = abs(elbow_angle - 180) if elbow_angle > 90 else elbow_angle
            wrist_angles.append(wrist_deviation)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize elbow angle
            shoulder_px = normalized_to_pixel_coordinates(shoulder[0], shoulder[1], frame_width, frame_height)
            elbow_px = normalized_to_pixel_coordinates(elbow[0], elbow[1], frame_width, frame_height)
            wrist_px = normalized_to_pixel_coordinates(wrist[0], wrist[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"Elbow: {elbow_angle:.1f}°",
                        (elbow_px[0] - 50, elbow_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine curl stage based on elbow angle
            if elbow_angle > 160 and curl_stage == "up":
                curl_stage = "down"
                # Draw curl stage
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # If we were in "down" position and now angle is < 60, we're in "up" position
            elif elbow_angle < 60 and (curl_stage == "down" or curl_stage is None):
                curl_stage = "up"
                curl_count += 1
                rep_depths.append(elbow_angle)  # Store depth of curl
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                print(f"Curl #{curl_count} detected at frame with elbow angle {elbow_angle:.1f}")
                
        # Display curl count
        cv2.putText(annotated_image, f'Curls: {curl_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path:
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "curl_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    avg_shoulder_stability = np.std(shoulder_stability) * 100  # Convert to percentage variation
    avg_wrist_deviation = np.mean(wrist_angles)
    avg_curl_depth = np.mean(rep_depths) if rep_depths else 0
    full_extension = np.max(elbow_angles) > 150
    
    # Generate feedback
    feedback = {
        "curl_count": curl_count,
        "form_analysis": {
            "curl_depth": avg_curl_depth,
            "full_extension": full_extension,
            "shoulder_stability": 100 - avg_shoulder_stability,  # Higher is better
            "wrist_stability": 100 - avg_wrist_deviation,  # Higher is better
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if avg_curl_depth > 45:
        feedback["feedback"].append("You're not curling the weight fully to your shoulder. Try to bring the weight closer to your shoulder.")
    
    if not full_extension:
        feedback["feedback"].append("You're not fully extending your arm at the bottom of the curl. Straighten your arm more for full range of motion.")
    
    if avg_shoulder_stability > 10:  # If shoulder position varies too much
        feedback["feedback"].append("Keep your upper arm and shoulder more stable during the curl. Avoid swinging.")
    
    if avg_wrist_deviation > 30:
        feedback["feedback"].append("Keep your wrist straight throughout the movement. Avoid bending your wrist.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your bicep curls show good depth, stable shoulders, and proper wrist position.")
        
    return feedback

# Example usage
result = analyze_bicep_curl("curl.MOV", "analyzed_curl.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('curl_count', 0)} curls")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Curl depth (lower is better): {result['form_analysis']['curl_depth']:.1f}°")
    print(f"Full arm extension: {'Yes' if result['form_analysis']['full_extension'] else 'No'}")
    print(f"Shoulder stability: {result['form_analysis']['shoulder_stability']:.1f}%")
    print(f"Wrist stability: {result['form_analysis']['wrist_stability']:.1f}%")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

Video dimensions: 0x0, FPS: 0

=== ANALYSIS RESULTS ===
Counted 0 curls
Error: Not enough valid pose detections. Check video quality and positioning.


I0000 00:00:1745945055.400436 6263613 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
OpenCV: Couldn't read video stream from file "curl.MOV"


W0000 00:00:1745945055.469340 6266840 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745945055.481120 6266840 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [5]:
%pip install mediapipe

  Using cached mediapipe-0.10.21-cp310-cp310-macosx_11_0_universal2.whl.metadata (9.9 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl.metadata (20 kB)
  Using cached sounddevice-0.5.1-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl.metadata (1.4 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 2.2 MB/s eta 0:00:0000:0100:01
Using cached flatbuffers-25.2.10-py2.py3-none-any.whl (30 kB)
Using cached sounddevice-0.5.1-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl (107 kB)
Using cached absl_py-2.2.2-py3-none-any.whl (135 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 MB 9.8 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [19]:
import mediapipe as mp
import cv2
import numpy as np
import math

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_bicep_curl(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track bicep curl state
    curl_count = 0
    curl_stage = None  # "up" or "down"
    good_frames = 0
    
    # Lists to store angles and positions for analysis
    elbow_angles = []
    shoulder_stability = []
    hip_stability = []
    wrist_angles = []
    rep_depths = []  # To track curl depth
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get key points for curl analysis
            # We'll focus on right arm for simplicity, but you could analyze both
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Get elbow angle (shoulder-elbow-wrist)
            elbow_angle = calculate_angle(shoulder, elbow, wrist)
            elbow_angles.append(elbow_angle)
            
            # Track shoulder stability - shoulder should remain still relative to hip
            shoulder_hip_distance = np.sqrt((shoulder[0] - hip[0])**2 + (shoulder[1] - hip[1])**2)
            shoulder_stability.append(shoulder_hip_distance)
            
            # Measure wrist angle relative to forearm (to detect wrist curling/improper form)
            # For simplicity, we'll estimate this as deviation from straight line
            wrist_deviation = abs(elbow_angle - 180) if elbow_angle > 90 else elbow_angle
            wrist_angles.append(wrist_deviation)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize elbow angle
            shoulder_px = normalized_to_pixel_coordinates(shoulder[0], shoulder[1], frame_width, frame_height)
            elbow_px = normalized_to_pixel_coordinates(elbow[0], elbow[1], frame_width, frame_height)
            wrist_px = normalized_to_pixel_coordinates(wrist[0], wrist[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"Elbow: {elbow_angle:.1f}°",
                        (elbow_px[0] - 50, elbow_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine curl stage based on elbow angle
            if elbow_angle > 160 and curl_stage == "up":
                curl_stage = "down"
                # Draw curl stage
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # If we were in "down" position and now angle is < 60, we're in "up" position
            elif elbow_angle < 60 and (curl_stage == "down" or curl_stage is None):
                curl_stage = "up"
                curl_count += 1
                rep_depths.append(elbow_angle)  # Store depth of curl
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                print(f"Curl #{curl_count} detected at frame with elbow angle {elbow_angle:.1f}")
                
        # Display curl count
        cv2.putText(annotated_image, f'Curls: {curl_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path:
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "curl_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    avg_shoulder_stability = np.std(shoulder_stability) * 100  # Convert to percentage variation
    avg_wrist_deviation = np.mean(wrist_angles)
    avg_curl_depth = np.mean(rep_depths) if rep_depths else 0
    full_extension = np.max(elbow_angles) > 150
    
    # Generate feedback
    feedback = {
        "curl_count": curl_count,
        "form_analysis": {
            "curl_depth": avg_curl_depth,
            "full_extension": full_extension,
            "shoulder_stability": 100 - avg_shoulder_stability,  # Higher is better
            "wrist_stability": 100 - avg_wrist_deviation,  # Higher is better
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if avg_curl_depth > 45:
        feedback["feedback"].append("You're not curling the weight fully to your shoulder. Try to bring the weight closer to your shoulder.")
    
    if not full_extension:
        feedback["feedback"].append("You're not fully extending your arm at the bottom of the curl. Straighten your arm more for full range of motion.")
    
    if avg_shoulder_stability > 10:  # If shoulder position varies too much
        feedback["feedback"].append("Keep your upper arm and shoulder more stable during the curl. Avoid swinging.")
    
    if avg_wrist_deviation > 30:
        feedback["feedback"].append("Keep your wrist straight throughout the movement. Avoid bending your wrist.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your bicep curls show good depth, stable shoulders, and proper wrist position.")
        
    return feedback

# Example usage
result = analyze_bicep_curl("clips/BicepCurl1.MOV", "clips/analyzed_curl1.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('curl_count', 0)} curls")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Curl depth (lower is better): {result['form_analysis']['curl_depth']:.1f}°")
    print(f"Full arm extension: {'Yes' if result['form_analysis']['full_extension'] else 'No'}")
    print(f"Shoulder stability: {result['form_analysis']['shoulder_stability']:.1f}%")
    print(f"Wrist stability: {result['form_analysis']['wrist_stability']:.1f}%")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745948767.804237 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745948767.894514 6365419 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745948767.909731 6365419 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29
Detected 33 landmarks
Curl #1 detected at frame with elbow angle 54.8
Curl #2 detected at frame with elbow angle 57.3

=== ANALYSIS RESULTS ===
Counted 2 curls
Curl depth (lower is better): 56.0°
Full arm extension: Yes
Shoulder stability: 99.7%
Wrist stability: 72.8%
Frames analyzed: 388

Feedback:
- You're not curling the weight fully to your shoulder. Try to bring the weight closer to your shoulder.


In [20]:
import mediapipe as mp
import cv2
import numpy as np
import math

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_bicep_curl(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track bicep curl state
    curl_count = 0
    curl_stage = None  # "up" or "down"
    good_frames = 0
    
    # Lists to store angles and positions for analysis
    elbow_angles = []
    shoulder_stability = []
    hip_stability = []
    wrist_angles = []
    rep_depths = []  # To track curl depth
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get key points for curl analysis
            # We'll focus on right arm for simplicity, but you could analyze both
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Get elbow angle (shoulder-elbow-wrist)
            elbow_angle = calculate_angle(shoulder, elbow, wrist)
            elbow_angles.append(elbow_angle)
            
            # Track shoulder stability - shoulder should remain still relative to hip
            shoulder_hip_distance = np.sqrt((shoulder[0] - hip[0])**2 + (shoulder[1] - hip[1])**2)
            shoulder_stability.append(shoulder_hip_distance)
            
            # Measure wrist angle relative to forearm (to detect wrist curling/improper form)
            # For simplicity, we'll estimate this as deviation from straight line
            wrist_deviation = abs(elbow_angle - 180) if elbow_angle > 90 else elbow_angle
            wrist_angles.append(wrist_deviation)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize elbow angle
            shoulder_px = normalized_to_pixel_coordinates(shoulder[0], shoulder[1], frame_width, frame_height)
            elbow_px = normalized_to_pixel_coordinates(elbow[0], elbow[1], frame_width, frame_height)
            wrist_px = normalized_to_pixel_coordinates(wrist[0], wrist[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"Elbow: {elbow_angle:.1f}°",
                        (elbow_px[0] - 50, elbow_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine curl stage based on elbow angle
            if elbow_angle > 160 and curl_stage == "up":
                curl_stage = "down"
                # Draw curl stage
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # If we were in "down" position and now angle is < 60, we're in "up" position
            elif elbow_angle < 60 and (curl_stage == "down" or curl_stage is None):
                curl_stage = "up"
                curl_count += 1
                rep_depths.append(elbow_angle)  # Store depth of curl
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                print(f"Curl #{curl_count} detected at frame with elbow angle {elbow_angle:.1f}")
                
        # Display curl count
        cv2.putText(annotated_image, f'Curls: {curl_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path:
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "curl_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    avg_shoulder_stability = np.std(shoulder_stability) * 100  # Convert to percentage variation
    avg_wrist_deviation = np.mean(wrist_angles)
    avg_curl_depth = np.mean(rep_depths) if rep_depths else 0
    full_extension = np.max(elbow_angles) > 150
    
    # Generate feedback
    feedback = {
        "curl_count": curl_count,
        "form_analysis": {
            "curl_depth": avg_curl_depth,
            "full_extension": full_extension,
            "shoulder_stability": 100 - avg_shoulder_stability,  # Higher is better
            "wrist_stability": 100 - avg_wrist_deviation,  # Higher is better
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if avg_curl_depth > 45:
        feedback["feedback"].append("You're not curling the weight fully to your shoulder. Try to bring the weight closer to your shoulder.")
    
    if not full_extension:
        feedback["feedback"].append("You're not fully extending your arm at the bottom of the curl. Straighten your arm more for full range of motion.")
    
    if avg_shoulder_stability > 10:  # If shoulder position varies too much
        feedback["feedback"].append("Keep your upper arm and shoulder more stable during the curl. Avoid swinging.")
    
    if avg_wrist_deviation > 30:
        feedback["feedback"].append("Keep your wrist straight throughout the movement. Avoid bending your wrist.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your bicep curls show good depth, stable shoulders, and proper wrist position.")
        
    return feedback

# Example usage
result = analyze_bicep_curl("clips/BicepCurl2.MOV", "clips/analyzed_curl2.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('curl_count', 0)} curls")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Curl depth (lower is better): {result['form_analysis']['curl_depth']:.1f}°")
    print(f"Full arm extension: {'Yes' if result['form_analysis']['full_extension'] else 'No'}")
    print(f"Shoulder stability: {result['form_analysis']['shoulder_stability']:.1f}%")
    print(f"Wrist stability: {result['form_analysis']['wrist_stability']:.1f}%")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745948799.276314 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745948799.348395 6366511 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745948799.358357 6366521 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29
Detected 33 landmarks
Curl #1 detected at frame with elbow angle 42.8

=== ANALYSIS RESULTS ===
Counted 1 curls
Curl depth (lower is better): 42.8°
Full arm extension: Yes
Shoulder stability: 99.0%
Wrist stability: 31.6%
Frames analyzed: 459

Feedback:
- Keep your wrist straight throughout the movement. Avoid bending your wrist.


In [ ]:
import mediapipe as mp
import cv2
import numpy as np
import math

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_bench_press(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track bench press state
    rep_count = 0
    press_stage = None  # "up" or "down"
    good_frames = 0
    
    # Lists to store angles and positions for analysis
    elbow_angles_left = []
    elbow_angles_right = []
    shoulder_width_values = []  # To track shoulder stability
    chest_depth_values = []  # To track chest engagement
    wrist_alignments = []  # To track wrist alignment with forearms
    rep_depths = []  # To track press depth
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get key points for bench press analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Also track chest (mid-point between shoulders) and hip landmarks for back arch analysis
            chest = [(left_shoulder[0] + right_shoulder[0])/2, 
                    (left_shoulder[1] + right_shoulder[1])/2]
            
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Get elbow angles (shoulder-elbow-wrist)
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            
            elbow_angles_left.append(left_elbow_angle)
            elbow_angles_right.append(right_elbow_angle)
            
            # Average elbow angle for determining press stage
            avg_elbow_angle = (left_elbow_angle + right_elbow_angle) / 2
            
            # Track shoulder width to ensure stable shoulder position during press
            shoulder_width = np.sqrt((left_shoulder[0] - right_shoulder[0])**2 + 
                                   (left_shoulder[1] - right_shoulder[1])**2)
            shoulder_width_values.append(shoulder_width)
            
            # Track chest depth (y-position of chest relative to shoulders)
            # Higher y value means lower position (due to image coordinate system)
            chest_depth = chest[1]  
            chest_depth_values.append(chest_depth)
            
            # Track wrist alignment (wrists should be aligned with forearms)
            left_wrist_alignment = abs(left_elbow_angle - 180) if left_elbow_angle > 90 else left_elbow_angle
            right_wrist_alignment = abs(right_elbow_angle - 180) if right_elbow_angle > 90 else right_elbow_angle
            wrist_alignments.append((left_wrist_alignment + right_wrist_alignment) / 2)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            left_elbow_px = normalized_to_pixel_coordinates(left_elbow[0], left_elbow[1], frame_width, frame_height)
            right_elbow_px = normalized_to_pixel_coordinates(right_elbow[0], right_elbow[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"L: {left_elbow_angle:.1f}°",
                        (left_elbow_px[0] - 50, left_elbow_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"R: {right_elbow_angle:.1f}°",
                        (right_elbow_px[0] - 50, right_elbow_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine press stage based on elbow angle
            # For bench press, "down" is when weights are near chest (elbow angle is small)
            # "up" is when arms are extended (elbow angle is large)
            if avg_elbow_angle > 160 and press_stage == "down":
                press_stage = "up"
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # If we were in "up" position and now angle is small, we're in "down" position
            elif avg_elbow_angle < 90 and (press_stage == "up" or press_stage is None):
                press_stage = "down"
                # If we're coming from "up", count a rep
                if press_stage == "up" or press_stage is None:
                    rep_count += 1
                    rep_depths.append(avg_elbow_angle)  # Store depth of press
                    print(f"Rep #{rep_count} detected at frame with average elbow angle {avg_elbow_angle:.1f}")
                
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path:
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    avg_shoulder_stability = np.std(shoulder_width_values) * 100  # Convert to percentage variation
    avg_wrist_alignment = np.mean(wrist_alignments)
    avg_press_depth = np.mean(rep_depths) if rep_depths else 0
    
    # Check for elbow flare (elbows should be at ~45° angle to body)
    elbow_symmetry = np.mean(np.abs(np.array(elbow_angles_left) - np.array(elbow_angles_right)))
    
    # Check for full extension at top of press
    full_extension = np.max(elbow_angles_left) > 160 and np.max(elbow_angles_right) > 160
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "press_depth": avg_press_depth,
            "full_extension": full_extension,
            "shoulder_stability": 100 - avg_shoulder_stability,  # Higher is better
            "wrist_alignment": 100 - avg_wrist_alignment,  # Higher is better
            "elbow_symmetry": 100 - elbow_symmetry,  # Higher is better
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if avg_press_depth > 70:  # Not going deep enough
        feedback["feedback"].append("You're not lowering the weights close enough to your chest. Try to achieve a deeper press for full muscle engagement.")
    
    if not full_extension:
        feedback["feedback"].append("You're not fully extending your arms at the top of the press. Straighten your arms more for full range of motion.")
    
    if avg_shoulder_stability > 10:  # If shoulder position varies too much
        feedback["feedback"].append("Keep your shoulders more stable during the press. Avoid excessive movement or shrugging.")
    
    if avg_wrist_alignment > 30:
        feedback["feedback"].append("Keep your wrists straight and aligned with your forearms. Avoid bending your wrists backward.")
    
    if elbow_symmetry > 20:
        feedback["feedback"].append("Your elbows are moving asymmetrically. Try to keep both arms moving together at the same pace and angle.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your bench press shows good depth, stable shoulders, proper wrist alignment, and symmetrical movement.")
        
    return feedback

# Example usage
result = analyze_bench_press("clips/BenchPress2.MOV", "clips/analyzed_bench_press.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Press depth (lower is better): {result['form_analysis']['press_depth']:.1f}°")
    print(f"Full arm extension: {'Yes' if result['form_analysis']['full_extension'] else 'No'}")
    print(f"Shoulder stability: {result['form_analysis']['shoulder_stability']:.1f}%")
    print(f"Wrist alignment: {result['form_analysis']['wrist_alignment']:.1f}%")
    print(f"Elbow symmetry: {result['form_analysis']['elbow_symmetry']:.1f}%")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745945101.329624 6263613 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745945101.420812 6268095 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745945101.432378 6268100 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745945101.446111 6268102 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Video dimensions: 1920x1080, FPS: 29
Detected 33 landmarks

=== ANALYSIS RESULTS ===
Counted 0 reps
Press depth (lower is better): 0.0°
Full arm extension: Yes
Shoulder stability: 98.1%
Wrist alignment: 57.2%
Elbow symmetry: 79.3%
Frames analyzed: 350

Feedback:
- Keep your wrists straight and aligned with your forearms. Avoid bending your wrists backward.
- Your elbows are moving asymmetrically. Try to keep both arms moving together at the same pace and angle.


In [11]:
import mediapipe as mp
import cv2
import numpy as np
import math

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_bench_press(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track bench press state
    rep_count = 0
    press_stage = None  # "up" or "down"
    good_frames = 0
    
    # Lists to store angles and positions for analysis
    elbow_angles_left = []
    elbow_angles_right = []
    shoulder_width_values = []  # To track shoulder stability
    chest_depth_values = []  # To track chest engagement
    wrist_alignments = []  # To track wrist alignment with forearms
    rep_depths = []  # To track press depth
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get key points for bench press analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Also track chest (mid-point between shoulders) and hip landmarks for back arch analysis
            chest = [(left_shoulder[0] + right_shoulder[0])/2, 
                    (left_shoulder[1] + right_shoulder[1])/2]
            
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Get elbow angles (shoulder-elbow-wrist)
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            
            elbow_angles_left.append(left_elbow_angle)
            elbow_angles_right.append(right_elbow_angle)
            
            # Average elbow angle for determining press stage
            avg_elbow_angle = (left_elbow_angle + right_elbow_angle) / 2
            
            # Track shoulder width to ensure stable shoulder position during press
            shoulder_width = np.sqrt((left_shoulder[0] - right_shoulder[0])**2 + 
                                   (left_shoulder[1] - right_shoulder[1])**2)
            shoulder_width_values.append(shoulder_width)
            
            # Track chest depth (y-position of chest relative to shoulders)
            # Higher y value means lower position (due to image coordinate system)
            chest_depth = chest[1]  
            chest_depth_values.append(chest_depth)
            
            # Track wrist alignment (wrists should be aligned with forearms)
            left_wrist_alignment = abs(left_elbow_angle - 180) if left_elbow_angle > 90 else left_elbow_angle
            right_wrist_alignment = abs(right_elbow_angle - 180) if right_elbow_angle > 90 else right_elbow_angle
            wrist_alignments.append((left_wrist_alignment + right_wrist_alignment) / 2)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            left_elbow_px = normalized_to_pixel_coordinates(left_elbow[0], left_elbow[1], frame_width, frame_height)
            right_elbow_px = normalized_to_pixel_coordinates(right_elbow[0], right_elbow[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"L: {left_elbow_angle:.1f}°",
                        (left_elbow_px[0] - 50, left_elbow_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"R: {right_elbow_angle:.1f}°",
                        (right_elbow_px[0] - 50, right_elbow_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine press stage based on elbow angle
            # For bench press, "down" is when weights are near chest (elbow angle is small)
            # "up" is when arms are extended (elbow angle is large)
            if avg_elbow_angle > 160 and press_stage == "down":
                press_stage = "up"
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # If we were in "up" position and now angle is small, we're in "down" position
            elif avg_elbow_angle < 90 and (press_stage == "up" or press_stage is None):
                press_stage = "down"
                # If we're coming from "up", count a rep
                if press_stage == "up" or press_stage is None:
                    rep_count += 1
                    rep_depths.append(avg_elbow_angle)  # Store depth of press
                    print(f"Rep #{rep_count} detected at frame with average elbow angle {avg_elbow_angle:.1f}")
                
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path:
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    avg_shoulder_stability = np.std(shoulder_width_values) * 100  # Convert to percentage variation
    avg_wrist_alignment = np.mean(wrist_alignments)
    avg_press_depth = np.mean(rep_depths) if rep_depths else 0
    
    # Check for elbow flare (elbows should be at ~45° angle to body)
    elbow_symmetry = np.mean(np.abs(np.array(elbow_angles_left) - np.array(elbow_angles_right)))
    
    # Check for full extension at top of press
    full_extension = np.max(elbow_angles_left) > 160 and np.max(elbow_angles_right) > 160
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "press_depth": avg_press_depth,
            "full_extension": full_extension,
            "shoulder_stability": 100 - avg_shoulder_stability,  # Higher is better
            "wrist_alignment": 100 - avg_wrist_alignment,  # Higher is better
            "elbow_symmetry": 100 - elbow_symmetry,  # Higher is better
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if avg_press_depth > 70:  # Not going deep enough
        feedback["feedback"].append("You're not lowering the weights close enough to your chest. Try to achieve a deeper press for full muscle engagement.")
    
    if not full_extension:
        feedback["feedback"].append("You're not fully extending your arms at the top of the press. Straighten your arms more for full range of motion.")
    
    if avg_shoulder_stability > 10:  # If shoulder position varies too much
        feedback["feedback"].append("Keep your shoulders more stable during the press. Avoid excessive movement or shrugging.")
    
    if avg_wrist_alignment > 30:
        feedback["feedback"].append("Keep your wrists straight and aligned with your forearms. Avoid bending your wrists backward.")
    
    if elbow_symmetry > 20:
        feedback["feedback"].append("Your elbows are moving asymmetrically. Try to keep both arms moving together at the same pace and angle.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your bench press shows good depth, stable shoulders, proper wrist alignment, and symmetrical movement.")
        
    return feedback

# Example usage
result = analyze_bench_press("clips/BenchPress1.MOV", "clips/analyzed_bench_press1.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Press depth (lower is better): {result['form_analysis']['press_depth']:.1f}°")
    print(f"Full arm extension: {'Yes' if result['form_analysis']['full_extension'] else 'No'}")
    print(f"Shoulder stability: {result['form_analysis']['shoulder_stability']:.1f}%")
    print(f"Wrist alignment: {result['form_analysis']['wrist_alignment']:.1f}%")
    print(f"Elbow symmetry: {result['form_analysis']['elbow_symmetry']:.1f}%")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745945194.285773 6263613 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745945194.366044 6270577 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745945194.379619 6270576 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29
Detected 33 landmarks

=== ANALYSIS RESULTS ===
Counted 0 reps
Press depth (lower is better): 0.0°
Full arm extension: Yes
Shoulder stability: 98.8%
Wrist alignment: 60.0%
Elbow symmetry: 64.6%
Frames analyzed: 301

Feedback:
- Keep your wrists straight and aligned with your forearms. Avoid bending your wrists backward.
- Your elbows are moving asymmetrically. Try to keep both arms moving together at the same pace and angle.


In [ ]:
import mediapipe as mp
import cv2
import numpy as np
import math
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_dumbbell_rows(video_path, output_video_path=None):
    # Check if file exists
    if not os.path.isfile(video_path):
        print(f"Error: File '{video_path}' does not exist!")
        return {
            "rep_count": 0,
            "error": f"Video file not found: {video_path}"
        }
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return {
            "rep_count": 0,
            "error": f"Could not open video file: {video_path}. Check file format and codecs."
        }
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {total_frames}")
    
    # If we still have 0x0 dimensions, there's a codec issue
    if frame_width == 0 or frame_height == 0:
        print(f"Error: Could not determine video dimensions. Try converting the video to MP4 format.")
        return {
            "rep_count": 0,
            "error": "Could not determine video dimensions. Try converting to MP4 format."
        }
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or try 'avc1'
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track row state
    rep_count = 0
    row_stage = None  # "up" (pulled up) or "down" (extended down)
    good_frames = 0
    
    # Lists to store angles and positions for analysis
    elbow_angles = []  # We'll focus primarily on one side for simplicity
    back_angles = []   # Track back angle relative to vertical
    hip_hinge_angles = [] # Track hip hinge
    shoulder_heights = [] # Track if shoulders stay level
    wrist_positions = []  # Track if wrist stays straight during pull
    rep_heights = []      # To track how high the elbow is pulled
    
    # Side detection - we'll determine which side the person is rowing with
    row_side = None       # "left" or "right"
    side_confidence = 0
    
    frame_count = 0
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get all key points for row analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            # Calculate spine midpoints for back angle
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, 
                          (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, 
                      (left_hip[1] + right_hip[1])/2]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Detect which side is rowing (first few frames)
            if good_frames <= 10 and row_side is None:
                # Compare elbow heights (y is inverted in image coordinates)
                left_elbow_height = left_elbow[1]
                right_elbow_height = right_elbow[1]
                
                # Lower y-value means higher position in image
                if left_elbow_height < right_elbow_height:
                    side_confidence += 1  # Left arm is higher
                else:
                    side_confidence -= 1  # Right arm is higher
                
                # After 10 frames, determine the side
                if good_frames == 10:
                    row_side = "left" if side_confidence > 0 else "right"
                    print(f"Detected rowing with {row_side} arm")
            
            # Choose which side to analyze based on detection
            if row_side == "left" or row_side is None:
                # Analyze left side
                active_shoulder = left_shoulder
                active_elbow = left_elbow
                active_wrist = left_wrist
                active_hip = left_hip
                display_prefix = "L"
            else:
                # Analyze right side
                active_shoulder = right_shoulder
                active_elbow = right_elbow
                active_wrist = right_wrist
                active_hip = right_hip
                display_prefix = "R"
            
            # Get elbow angle (shoulder-elbow-wrist)
            elbow_angle = calculate_angle(active_shoulder, active_elbow, active_wrist)
            elbow_angles.append(elbow_angle)
            
            # Calculate back angle relative to vertical
            # Create a point directly below mid_hip to represent vertical
            vertical_point = [mid_hip[0], mid_hip[1] + 0.2]  # Adding to y moves down in image coords
            back_angle = calculate_angle(mid_shoulder, mid_hip, vertical_point)
            # Convert to angle from vertical (90° would be perfectly upright)
            back_angle = 180 - back_angle if back_angle > 90 else back_angle
            back_angles.append(back_angle)
            
            # Calculate hip hinge (knee-hip-shoulder angle)
            # For left side
            left_hip_angle = calculate_angle(left_knee, left_hip, left_shoulder)
            # For right side
            right_hip_angle = calculate_angle(right_knee, right_hip, right_shoulder)
            # Use the side we're analyzing
            hip_angle = left_hip_angle if row_side == "left" else right_hip_angle
            hip_hinge_angles.append(hip_angle)
            
            # Track shoulder level (should stay parallel to ground during rows)
            shoulder_height_diff = abs(left_shoulder[1] - right_shoulder[1])
            shoulder_heights.append(shoulder_height_diff)
            
            # Track wrist position relative to elbow (should stay in line)
            # For simplicity, we'll use the elbow angle as a proxy for wrist alignment
            wrist_positions.append(elbow_angle)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            active_elbow_px = normalized_to_pixel_coordinates(active_elbow[0], active_elbow[1], frame_width, frame_height)
            mid_hip_px = normalized_to_pixel_coordinates(mid_hip[0], mid_hip[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"Elbow: {elbow_angle:.1f}°",
                        (active_elbow_px[0] - 50, active_elbow_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Back: {back_angle:.1f}°",
                        (mid_hip_px[0] - 50, mid_hip_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine row stage based on elbow angle
            # For rows, "down" is when arm is extended down/forward
            # "up" is when elbow is pulled back/up
            
            # Row is "up" when elbow angle is small (arm bent in pulling position)
            if elbow_angle < 90 and row_stage == "down":
                row_stage = "up"
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                rep_heights.append(active_elbow[1])  # Track height of the pull (y-coordinate)
            
            # Row is "down" when elbow angle is large (arm extended)
            elif elbow_angle > 150 and (row_stage == "up" or row_stage is None):
                row_stage = "down"
                # If coming from "up", count a rep
                if row_stage == "up":
                    rep_count += 1
                    print(f"Rep #{rep_count} detected at frame with elbow angle {elbow_angle:.1f}")
                
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path and 'out' in locals():
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    avg_back_angle = np.mean(back_angles) if back_angles else 0
    avg_hip_hinge = np.mean(hip_hinge_angles) if hip_hinge_angles else 0
    shoulder_stability = np.std(shoulder_heights) * 100 if shoulder_heights else 0
    wrist_alignment = np.mean(abs(np.array(wrist_positions) - 180)) if wrist_positions else 0
    
    # Check for full extension at bottom of row
    full_extension = np.max(elbow_angles) > 150 if elbow_angles else False
    
    # Check for adequate pull (elbow should be pulled back enough)
    adequate_pull = np.min(elbow_angles) < 80 if elbow_angles else False
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "back_angle": avg_back_angle,  # Closer to 45° is ideal for rows
            "hip_hinge": avg_hip_hinge,    # Should be around 170-175° for proper hip hinge
            "full_extension": full_extension,
            "adequate_pull": adequate_pull,
            "shoulder_stability": 100 - shoulder_stability,  # Higher is better
            "wrist_alignment": 100 - wrist_alignment,        # Higher is better
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    # Check back angle - should be around 45° (bent forward but not too much)
    if avg_back_angle < 30:
        feedback["feedback"].append("Your back is too vertical. Bend forward more from the hips to engage your back muscles properly.")
    elif avg_back_angle > 60:
        feedback["feedback"].append("You're bending too far forward. Maintain a back angle around 45° to protect your lower back.")
    
    # Check hip hinge - should show proper hinging
    if avg_hip_hinge < 160:
        feedback["feedback"].append("Improve your hip hinge. Bend at the hips rather than rounding your back.")
    
    # Check pull height/quality
    if not adequate_pull:
        feedback["feedback"].append("Pull the dumbbell higher by bringing your elbow further back to fully engage your back muscles.")
    
    # Check extension
    if not full_extension:
        feedback["feedback"].append("Fully extend your arm at the bottom of the row for complete range of motion.")
    
    # Check shoulder stability
    if shoulder_stability > 10:
        feedback["feedback"].append("Keep your shoulders level throughout the movement. Avoid twisting or rotating your torso.")
    
    # Check wrist alignment
    if wrist_alignment > 30:
        feedback["feedback"].append("Keep your wrist straight and aligned with your forearm throughout the row.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your dumbbell rows show good back angle, proper hip hinge, full range of motion, and good control.")
        
    return feedback

# Example usage - Make sure you have the correct file path
video_file = "clips/DumbbellRow2.MOV"  # Update this to the correct path
result = analyze_dumbbell_rows(video_file, "clips/analyzed_row2.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Back angle (ideal ~45°): {result['form_analysis']['back_angle']:.1f}°")
    print(f"Hip hinge (ideal ~170-175°): {result['form_analysis']['hip_hinge']:.1f}°")
    print(f"Full arm extension: {'Yes' if result['form_analysis']['full_extension'] else 'No'}")
    print(f"Adequate pull: {'Yes' if result['form_analysis']['adequate_pull'] else 'No'}")
    print(f"Shoulder stability: {result['form_analysis']['shoulder_stability']:.1f}%")
    print(f"Wrist alignment: {result['form_analysis']['wrist_alignment']:.1f}%")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745947331.246154 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745947331.368179 6322569 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745947331.380640 6322569 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745947331.395203 6322577 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Video dimensions: 1920x1080, FPS: 29, Total frames: 455
Detected 33 landmarks
Detected rowing with right arm
Processing frame 100/455
Processing frame 200/455
Processing frame 300/455
Processing frame 400/455

=== ANALYSIS RESULTS ===
Counted 0 reps
Back angle (ideal ~45°): 9.9°
Hip hinge (ideal ~170-175°): 68.2°
Full arm extension: Yes
Adequate pull: Yes
Shoulder stability: 96.4%
Wrist alignment: 73.0%
Frames analyzed: 455

Feedback:
- Your back is too vertical. Bend forward more from the hips to engage your back muscles properly.
- Improve your hip hinge. Bend at the hips rather than rounding your back.


In [5]:
import mediapipe as mp
import cv2
import numpy as np
import math
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_dumbbell_rows(video_path, output_video_path=None):
    # Check if file exists
    if not os.path.isfile(video_path):
        print(f"Error: File '{video_path}' does not exist!")
        return {
            "rep_count": 0,
            "error": f"Video file not found: {video_path}"
        }
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return {
            "rep_count": 0,
            "error": f"Could not open video file: {video_path}. Check file format and codecs."
        }
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {total_frames}")
    
    # If we still have 0x0 dimensions, there's a codec issue
    if frame_width == 0 or frame_height == 0:
        print(f"Error: Could not determine video dimensions. Try converting the video to MP4 format.")
        return {
            "rep_count": 0,
            "error": "Could not determine video dimensions. Try converting to MP4 format."
        }
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or try 'avc1'
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track row state
    rep_count = 0
    row_stage = None  # "up" (pulled up) or "down" (extended down)
    good_frames = 0
    
    # Lists to store angles and positions for analysis
    elbow_angles = []  # We'll focus primarily on one side for simplicity
    back_angles = []   # Track back angle relative to vertical
    hip_hinge_angles = [] # Track hip hinge
    shoulder_heights = [] # Track if shoulders stay level
    wrist_positions = []  # Track if wrist stays straight during pull
    rep_heights = []      # To track how high the elbow is pulled
    
    # Side detection - we'll determine which side the person is rowing with
    row_side = None       # "left" or "right"
    side_confidence = 0
    
    frame_count = 0
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get all key points for row analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            # Calculate spine midpoints for back angle
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, 
                          (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, 
                      (left_hip[1] + right_hip[1])/2]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Detect which side is rowing (first few frames)
            if good_frames <= 10 and row_side is None:
                # Compare elbow heights (y is inverted in image coordinates)
                left_elbow_height = left_elbow[1]
                right_elbow_height = right_elbow[1]
                
                # Lower y-value means higher position in image
                if left_elbow_height < right_elbow_height:
                    side_confidence += 1  # Left arm is higher
                else:
                    side_confidence -= 1  # Right arm is higher
                
                # After 10 frames, determine the side
                if good_frames == 10:
                    row_side = "left" if side_confidence > 0 else "right"
                    print(f"Detected rowing with {row_side} arm")
            
            # Choose which side to analyze based on detection
            if row_side == "left" or row_side is None:
                # Analyze left side
                active_shoulder = left_shoulder
                active_elbow = left_elbow
                active_wrist = left_wrist
                active_hip = left_hip
                display_prefix = "L"
            else:
                # Analyze right side
                active_shoulder = right_shoulder
                active_elbow = right_elbow
                active_wrist = right_wrist
                active_hip = right_hip
                display_prefix = "R"
            
            # Get elbow angle (shoulder-elbow-wrist)
            elbow_angle = calculate_angle(active_shoulder, active_elbow, active_wrist)
            elbow_angles.append(elbow_angle)
            
            # Calculate back angle relative to vertical
            # Create a point directly below mid_hip to represent vertical
            vertical_point = [mid_hip[0], mid_hip[1] + 0.2]  # Adding to y moves down in image coords
            back_angle = calculate_angle(mid_shoulder, mid_hip, vertical_point)
            # Convert to angle from vertical (90° would be perfectly upright)
            back_angle = 180 - back_angle if back_angle > 90 else back_angle
            back_angles.append(back_angle)
            
            # Calculate hip hinge (knee-hip-shoulder angle)
            # For left side
            left_hip_angle = calculate_angle(left_knee, left_hip, left_shoulder)
            # For right side
            right_hip_angle = calculate_angle(right_knee, right_hip, right_shoulder)
            # Use the side we're analyzing
            hip_angle = left_hip_angle if row_side == "left" else right_hip_angle
            hip_hinge_angles.append(hip_angle)
            
            # Track shoulder level (should stay parallel to ground during rows)
            shoulder_height_diff = abs(left_shoulder[1] - right_shoulder[1])
            shoulder_heights.append(shoulder_height_diff)
            
            # Track wrist position relative to elbow (should stay in line)
            # For simplicity, we'll use the elbow angle as a proxy for wrist alignment
            wrist_positions.append(elbow_angle)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            active_elbow_px = normalized_to_pixel_coordinates(active_elbow[0], active_elbow[1], frame_width, frame_height)
            mid_hip_px = normalized_to_pixel_coordinates(mid_hip[0], mid_hip[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"Elbow: {elbow_angle:.1f}°",
                        (active_elbow_px[0] - 50, active_elbow_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Back: {back_angle:.1f}°",
                        (mid_hip_px[0] - 50, mid_hip_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine row stage based on elbow angle
            # For rows, "down" is when arm is extended down/forward
            # "up" is when elbow is pulled back/up
            
            # Row is "up" when elbow angle is small (arm bent in pulling position)
            if elbow_angle < 90 and row_stage == "down":
                row_stage = "up"
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                rep_heights.append(active_elbow[1])  # Track height of the pull (y-coordinate)
            
            # Row is "down" when elbow angle is large (arm extended)
            elif elbow_angle > 150 and (row_stage == "up" or row_stage is None):
                row_stage = "down"
                # If coming from "up", count a rep
                if row_stage == "up":
                    rep_count += 1
                    print(f"Rep #{rep_count} detected at frame with elbow angle {elbow_angle:.1f}")
                
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path and 'out' in locals():
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    avg_back_angle = np.mean(back_angles) if back_angles else 0
    avg_hip_hinge = np.mean(hip_hinge_angles) if hip_hinge_angles else 0
    shoulder_stability = np.std(shoulder_heights) * 100 if shoulder_heights else 0
    wrist_alignment = np.mean(abs(np.array(wrist_positions) - 180)) if wrist_positions else 0
    
    # Check for full extension at bottom of row
    full_extension = np.max(elbow_angles) > 150 if elbow_angles else False
    
    # Check for adequate pull (elbow should be pulled back enough)
    adequate_pull = np.min(elbow_angles) < 80 if elbow_angles else False
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "back_angle": avg_back_angle,  # Closer to 45° is ideal for rows
            "hip_hinge": avg_hip_hinge,    # Should be around 170-175° for proper hip hinge
            "full_extension": full_extension,
            "adequate_pull": adequate_pull,
            "shoulder_stability": 100 - shoulder_stability,  # Higher is better
            "wrist_alignment": 100 - wrist_alignment,        # Higher is better
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    # Check back angle - should be around 45° (bent forward but not too much)
    if avg_back_angle < 30:
        feedback["feedback"].append("Your back is too vertical. Bend forward more from the hips to engage your back muscles properly.")
    elif avg_back_angle > 60:
        feedback["feedback"].append("You're bending too far forward. Maintain a back angle around 45° to protect your lower back.")
    
    # Check hip hinge - should show proper hinging
    if avg_hip_hinge < 160:
        feedback["feedback"].append("Improve your hip hinge. Bend at the hips rather than rounding your back.")
    
    # Check pull height/quality
    if not adequate_pull:
        feedback["feedback"].append("Pull the dumbbell higher by bringing your elbow further back to fully engage your back muscles.")
    
    # Check extension
    if not full_extension:
        feedback["feedback"].append("Fully extend your arm at the bottom of the row for complete range of motion.")
    
    # Check shoulder stability
    if shoulder_stability > 10:
        feedback["feedback"].append("Keep your shoulders level throughout the movement. Avoid twisting or rotating your torso.")
    
    # Check wrist alignment
    if wrist_alignment > 30:
        feedback["feedback"].append("Keep your wrist straight and aligned with your forearm throughout the row.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your dumbbell rows show good back angle, proper hip hinge, full range of motion, and good control.")
        
    return feedback

# Example usage - Make sure you have the correct file path
video_file = "clips/DumbbellRow1.MOV"  # Update this to the correct path
result = analyze_dumbbell_rows(video_file, "clips/analyzed_row1.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Back angle (ideal ~45°): {result['form_analysis']['back_angle']:.1f}°")
    print(f"Hip hinge (ideal ~170-175°): {result['form_analysis']['hip_hinge']:.1f}°")
    print(f"Full arm extension: {'Yes' if result['form_analysis']['full_extension'] else 'No'}")
    print(f"Adequate pull: {'Yes' if result['form_analysis']['adequate_pull'] else 'No'}")
    print(f"Shoulder stability: {result['form_analysis']['shoulder_stability']:.1f}%")
    print(f"Wrist alignment: {result['form_analysis']['wrist_alignment']:.1f}%")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745947431.435876 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745947431.544536 6326472 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745947431.560739 6326479 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29, Total frames: 331
Detected 33 landmarks
Detected rowing with left arm
Processing frame 100/331
Processing frame 200/331
Processing frame 300/331

=== ANALYSIS RESULTS ===
Counted 0 reps
Back angle (ideal ~45°): 15.4°
Hip hinge (ideal ~170-175°): 88.4°
Full arm extension: Yes
Adequate pull: Yes
Shoulder stability: 99.1%
Wrist alignment: 40.3%
Frames analyzed: 329

Feedback:
- Your back is too vertical. Bend forward more from the hips to engage your back muscles properly.
- Improve your hip hinge. Bend at the hips rather than rounding your back.
- Keep your wrist straight and aligned with your forearm throughout the row.


In [6]:
import mediapipe as mp
import cv2
import numpy as np
import math
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_goblet_squat(video_path, output_video_path=None):
    # Check if file exists
    if not os.path.isfile(video_path):
        print(f"Error: File '{video_path}' does not exist!")
        return {
            "rep_count": 0,
            "error": f"Video file not found: {video_path}"
        }
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return {
            "rep_count": 0,
            "error": f"Could not open video file: {video_path}. Check file format and codecs."
        }
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {total_frames}")
    
    # If we still have 0x0 dimensions, there's a codec issue
    if frame_width == 0 or frame_height == 0:
        print(f"Error: Could not determine video dimensions. Try converting the video to MP4 format.")
        return {
            "rep_count": 0,
            "error": "Could not determine video dimensions. Try converting to MP4 format."
        }
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or try 'avc1'
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track squat state
    rep_count = 0
    squat_stage = None  # "up" (standing) or "down" (squatting)
    good_frames = 0
    
    # Lists to store angles and positions for analysis
    knee_angles_left = []
    knee_angles_right = []
    hip_angles_left = []
    hip_angles_right = []
    ankle_angles_left = []
    ankle_angles_right = []
    back_angles = []
    hip_depths = []
    knee_alignments = []  # Track knee alignment with ankles and hips
    torso_heights = []    # Track torso height for squat depth
    
    frame_count = 0
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get all key points for squat analysis
            # Foot points
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            left_heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]
            left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_heel = [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y]
            right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
            
            # Leg points
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Upper body points
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            
            # Calculate spine midpoints for back angle
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, 
                          (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, 
                      (left_hip[1] + right_hip[1])/2]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Calculate knee angles (hip-knee-ankle)
            left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            
            knee_angles_left.append(left_knee_angle)
            knee_angles_right.append(right_knee_angle)
            
            # Calculate hip angles (knee-hip-shoulder)
            left_hip_angle = calculate_angle(left_knee, left_hip, left_shoulder)
            right_hip_angle = calculate_angle(right_knee, right_hip, right_shoulder)
            
            hip_angles_left.append(left_hip_angle)
            hip_angles_right.append(right_hip_angle)
            
            # Calculate ankle angles (knee-ankle-foot_index) - measures dorsiflexion
            left_ankle_angle = calculate_angle(left_knee, left_ankle, left_foot_index)
            right_ankle_angle = calculate_angle(right_knee, right_ankle, right_foot_index)
            
            ankle_angles_left.append(left_ankle_angle)
            ankle_angles_right.append(right_ankle_angle)
            
            # Calculate back angle relative to vertical
            # Create a point directly below mid_hip to represent vertical
            vertical_point = [mid_hip[0], mid_hip[1] + 0.2]  # Adding to y moves down in image coords
            back_angle = calculate_angle(mid_shoulder, mid_hip, vertical_point)
            back_angle = 180 - back_angle if back_angle > 90 else back_angle
            back_angles.append(back_angle)
            
            # Track hip depth (y-position of hip relative to knee)
            # In a proper squat, hips should drop to at least knee level or slightly below
            left_hip_depth = left_hip[1] - left_knee[1]  # Negative values mean hip is above knee
            right_hip_depth = right_hip[1] - right_knee[1]
            avg_hip_depth = (left_hip_depth + right_hip_depth) / 2
            hip_depths.append(avg_hip_depth)
            
            # Calculate knee alignment
            # Measure horizontal distance from knee to ankle to track knee position
            # Knees should track over toes, not cave in or out
            left_knee_to_ankle_x = left_knee[0] - left_ankle[0]
            right_knee_to_ankle_x = right_knee[0] - right_ankle[0]
            knee_alignments.append((left_knee_to_ankle_x, right_knee_to_ankle_x))
            
            # Track torso height for squat depth
            torso_height = mid_shoulder[1]  # y-coordinate of mid-shoulder
            torso_heights.append(torso_height)
            
            # Average knee angle for determining squat stage
            avg_knee_angle = (left_knee_angle + right_knee_angle) / 2
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            left_knee_px = normalized_to_pixel_coordinates(left_knee[0], left_knee[1], frame_width, frame_height)
            right_knee_px = normalized_to_pixel_coordinates(right_knee[0], right_knee[1], frame_width, frame_height)
            mid_hip_px = normalized_to_pixel_coordinates(mid_hip[0], mid_hip[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"L Knee: {left_knee_angle:.1f}°",
                        (left_knee_px[0] - 70, left_knee_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"R Knee: {right_knee_angle:.1f}°",
                        (right_knee_px[0] - 70, right_knee_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Back: {back_angle:.1f}°",
                        (mid_hip_px[0] - 50, mid_hip_px[1] - 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine squat stage based on knee angle
            # For squats, "down" is when knees are deeply bent (angle is small)
            # "up" is when standing (angle is large close to 180)
            
            # Squat is "down" when knee angle is small (deep squat position)
            if avg_knee_angle < 110 and (squat_stage == "up" or squat_stage is None):
                squat_stage = "down"
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # Squat is "up" when knee angle is large (standing position)
            elif avg_knee_angle > 160 and squat_stage == "down":
                squat_stage = "up"
                # Coming from "down", count a rep
                rep_count += 1
                print(f"Rep #{rep_count} detected at frame with average knee angle {avg_knee_angle:.1f}")
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path and 'out' in locals():
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    avg_knee_angle_min = np.min([np.min(knee_angles_left), np.min(knee_angles_right)]) if knee_angles_left and knee_angles_right else 180
    avg_hip_angle_min = np.min([np.min(hip_angles_left), np.min(hip_angles_right)]) if hip_angles_left and hip_angles_right else 180
    avg_ankle_angle = np.mean([np.mean(ankle_angles_left), np.mean(ankle_angles_right)]) if ankle_angles_left and ankle_angles_right else 0
    
    avg_back_angle = np.mean(back_angles) if back_angles else 90
    max_hip_depth = np.max(hip_depths) if hip_depths else 0
    
    # Calculate knee alignment score
    # Ideal is when knees track over ankles
    knee_alignment_scores = []
    for left_align, right_align in knee_alignments:
        # Calculate a score where 0 is perfect alignment
        # Higher values mean knees are tracking too far in or out
        left_score = abs(left_align)
        right_score = abs(right_align)
        knee_alignment_scores.append((left_score + right_score) / 2)
    
    avg_knee_alignment = np.mean(knee_alignment_scores) if knee_alignment_scores else 0
    
    # Full squat check - did they go deep enough?
    deep_squat = avg_knee_angle_min < 100  # Proper depth typically has knee angle < 100°
    
    # Check for knees passing toes
    # This is a simplified check - in reality, the judgment depends on ankle mobility
    knees_past_toes = False
    if knee_alignments:
        # Check if there are many frames where knees are significantly ahead of ankles
        forward_knee_count = sum(1 for l, r in knee_alignments if abs(l) > 0.1 or abs(r) > 0.1)
        knees_past_toes = forward_knee_count > len(knee_alignments) * 0.5
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "squat_depth": avg_knee_angle_min,           # Lower is deeper
            "hip_mobility": avg_hip_angle_min,           # Lower shows better hip mobility
            "ankle_mobility": avg_ankle_angle,           # Higher angle indicates better dorsiflexion
            "back_angle": avg_back_angle,                # Should stay upright (close to 90°)
            "knee_alignment": 100 - avg_knee_alignment*100,  # Higher is better (knees tracking over toes)
            "deep_squat": deep_squat,
            "knees_past_toes": knees_past_toes,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    # Check squat depth
    if not deep_squat:
        feedback["feedback"].append("Try to squat deeper. Aim to get your thighs at least parallel to the ground for full muscle engagement.")
    
    # Check back position
    if avg_back_angle < 70:
        feedback["feedback"].append("Try to keep your back more upright. Your torso is leaning too far forward during the squat.")
    
    # Check knee alignment
    if avg_knee_alignment > 0.1:
        feedback["feedback"].append("Focus on keeping your knees aligned with your toes. Your knees are tracking too far inward or outward.")
    
    # Check for knees past toes
    if knees_past_toes:
        feedback["feedback"].append("Your knees are moving too far forward past your toes. Try to push your hips back more as you descend.")
    
    # Check ankle mobility
    if avg_ankle_angle < 70:
        feedback["feedback"].append("Work on ankle mobility. Limited ankle dorsiflexion is affecting your squat depth and form.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your goblet squat shows good depth, proper back position, and good knee alignment.")
        
    return feedback

# Example usage - Make sure you have the correct file path
video_file = "clips/GobletSquat1.MOV"  # Update this to the correct path
result = analyze_goblet_squat(video_file, "clips/analyzed_squat1.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Squat depth (knee angle, lower is deeper): {result['form_analysis']['squat_depth']:.1f}°")
    print(f"Hip mobility (hip angle, lower is better): {result['form_analysis']['hip_mobility']:.1f}°")
    print(f"Ankle mobility: {result['form_analysis']['ankle_mobility']:.1f}°")
    print(f"Back angle (90° is upright): {result['form_analysis']['back_angle']:.1f}°")
    print(f"Knee alignment score: {result['form_analysis']['knee_alignment']:.1f}%")
    print(f"Achieved full depth: {'Yes' if result['form_analysis']['deep_squat'] else 'No'}")
    print(f"Knees past toes: {'Yes' if result['form_analysis']['knees_past_toes'] else 'No'}")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745947599.568750 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745947599.675835 6331069 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745947599.691168 6331068 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29, Total frames: 549
Detected 33 landmarks
Rep #1 detected at frame with average knee angle 165.7
Processing frame 100/549
Processing frame 200/549
Rep #2 detected at frame with average knee angle 161.7
Processing frame 300/549
Rep #3 detected at frame with average knee angle 162.1
Rep #4 detected at frame with average knee angle 161.3
Processing frame 400/549
Rep #5 detected at frame with average knee angle 163.1
Processing frame 500/549
Rep #6 detected at frame with average knee angle 163.1

=== ANALYSIS RESULTS ===
Counted 6 reps
Squat depth (knee angle, lower is deeper): 79.0°
Hip mobility (hip angle, lower is better): 85.7°
Ankle mobility: 159.9°
Back angle (90° is upright): 87.8°
Knee alignment score: 90.4%
Achieved full depth: Yes
Knees past toes: Yes
Frames analyzed: 549

Feedback:
- Your knees are moving too far forward past your toes. Try to push your hips back more as you descend.


In [7]:
import mediapipe as mp
import cv2
import numpy as np
import math
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_goblet_squat(video_path, output_video_path=None):
    # Check if file exists
    if not os.path.isfile(video_path):
        print(f"Error: File '{video_path}' does not exist!")
        return {
            "rep_count": 0,
            "error": f"Video file not found: {video_path}"
        }
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return {
            "rep_count": 0,
            "error": f"Could not open video file: {video_path}. Check file format and codecs."
        }
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {total_frames}")
    
    # If we still have 0x0 dimensions, there's a codec issue
    if frame_width == 0 or frame_height == 0:
        print(f"Error: Could not determine video dimensions. Try converting the video to MP4 format.")
        return {
            "rep_count": 0,
            "error": "Could not determine video dimensions. Try converting to MP4 format."
        }
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or try 'avc1'
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track squat state
    rep_count = 0
    squat_stage = None  # "up" (standing) or "down" (squatting)
    good_frames = 0
    
    # Lists to store angles and positions for analysis
    knee_angles_left = []
    knee_angles_right = []
    hip_angles_left = []
    hip_angles_right = []
    ankle_angles_left = []
    ankle_angles_right = []
    back_angles = []
    hip_depths = []
    knee_alignments = []  # Track knee alignment with ankles and hips
    torso_heights = []    # Track torso height for squat depth
    
    frame_count = 0
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get all key points for squat analysis
            # Foot points
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            left_heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]
            left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_heel = [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y]
            right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
            
            # Leg points
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Upper body points
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            
            # Calculate spine midpoints for back angle
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, 
                          (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, 
                      (left_hip[1] + right_hip[1])/2]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Calculate knee angles (hip-knee-ankle)
            left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            
            knee_angles_left.append(left_knee_angle)
            knee_angles_right.append(right_knee_angle)
            
            # Calculate hip angles (knee-hip-shoulder)
            left_hip_angle = calculate_angle(left_knee, left_hip, left_shoulder)
            right_hip_angle = calculate_angle(right_knee, right_hip, right_shoulder)
            
            hip_angles_left.append(left_hip_angle)
            hip_angles_right.append(right_hip_angle)
            
            # Calculate ankle angles (knee-ankle-foot_index) - measures dorsiflexion
            left_ankle_angle = calculate_angle(left_knee, left_ankle, left_foot_index)
            right_ankle_angle = calculate_angle(right_knee, right_ankle, right_foot_index)
            
            ankle_angles_left.append(left_ankle_angle)
            ankle_angles_right.append(right_ankle_angle)
            
            # Calculate back angle relative to vertical
            # Create a point directly below mid_hip to represent vertical
            vertical_point = [mid_hip[0], mid_hip[1] + 0.2]  # Adding to y moves down in image coords
            back_angle = calculate_angle(mid_shoulder, mid_hip, vertical_point)
            back_angle = 180 - back_angle if back_angle > 90 else back_angle
            back_angles.append(back_angle)
            
            # Track hip depth (y-position of hip relative to knee)
            # In a proper squat, hips should drop to at least knee level or slightly below
            left_hip_depth = left_hip[1] - left_knee[1]  # Negative values mean hip is above knee
            right_hip_depth = right_hip[1] - right_knee[1]
            avg_hip_depth = (left_hip_depth + right_hip_depth) / 2
            hip_depths.append(avg_hip_depth)
            
            # Calculate knee alignment
            # Measure horizontal distance from knee to ankle to track knee position
            # Knees should track over toes, not cave in or out
            left_knee_to_ankle_x = left_knee[0] - left_ankle[0]
            right_knee_to_ankle_x = right_knee[0] - right_ankle[0]
            knee_alignments.append((left_knee_to_ankle_x, right_knee_to_ankle_x))
            
            # Track torso height for squat depth
            torso_height = mid_shoulder[1]  # y-coordinate of mid-shoulder
            torso_heights.append(torso_height)
            
            # Average knee angle for determining squat stage
            avg_knee_angle = (left_knee_angle + right_knee_angle) / 2
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            left_knee_px = normalized_to_pixel_coordinates(left_knee[0], left_knee[1], frame_width, frame_height)
            right_knee_px = normalized_to_pixel_coordinates(right_knee[0], right_knee[1], frame_width, frame_height)
            mid_hip_px = normalized_to_pixel_coordinates(mid_hip[0], mid_hip[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"L Knee: {left_knee_angle:.1f}°",
                        (left_knee_px[0] - 70, left_knee_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"R Knee: {right_knee_angle:.1f}°",
                        (right_knee_px[0] - 70, right_knee_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Back: {back_angle:.1f}°",
                        (mid_hip_px[0] - 50, mid_hip_px[1] - 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine squat stage based on knee angle
            # For squats, "down" is when knees are deeply bent (angle is small)
            # "up" is when standing (angle is large close to 180)
            
            # Squat is "down" when knee angle is small (deep squat position)
            if avg_knee_angle < 110 and (squat_stage == "up" or squat_stage is None):
                squat_stage = "down"
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # Squat is "up" when knee angle is large (standing position)
            elif avg_knee_angle > 160 and squat_stage == "down":
                squat_stage = "up"
                # Coming from "down", count a rep
                rep_count += 1
                print(f"Rep #{rep_count} detected at frame with average knee angle {avg_knee_angle:.1f}")
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path and 'out' in locals():
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    avg_knee_angle_min = np.min([np.min(knee_angles_left), np.min(knee_angles_right)]) if knee_angles_left and knee_angles_right else 180
    avg_hip_angle_min = np.min([np.min(hip_angles_left), np.min(hip_angles_right)]) if hip_angles_left and hip_angles_right else 180
    avg_ankle_angle = np.mean([np.mean(ankle_angles_left), np.mean(ankle_angles_right)]) if ankle_angles_left and ankle_angles_right else 0
    
    avg_back_angle = np.mean(back_angles) if back_angles else 90
    max_hip_depth = np.max(hip_depths) if hip_depths else 0
    
    # Calculate knee alignment score
    # Ideal is when knees track over ankles
    knee_alignment_scores = []
    for left_align, right_align in knee_alignments:
        # Calculate a score where 0 is perfect alignment
        # Higher values mean knees are tracking too far in or out
        left_score = abs(left_align)
        right_score = abs(right_align)
        knee_alignment_scores.append((left_score + right_score) / 2)
    
    avg_knee_alignment = np.mean(knee_alignment_scores) if knee_alignment_scores else 0
    
    # Full squat check - did they go deep enough?
    deep_squat = avg_knee_angle_min < 100  # Proper depth typically has knee angle < 100°
    
    # Check for knees passing toes
    # This is a simplified check - in reality, the judgment depends on ankle mobility
    knees_past_toes = False
    if knee_alignments:
        # Check if there are many frames where knees are significantly ahead of ankles
        forward_knee_count = sum(1 for l, r in knee_alignments if abs(l) > 0.1 or abs(r) > 0.1)
        knees_past_toes = forward_knee_count > len(knee_alignments) * 0.5
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "squat_depth": avg_knee_angle_min,           # Lower is deeper
            "hip_mobility": avg_hip_angle_min,           # Lower shows better hip mobility
            "ankle_mobility": avg_ankle_angle,           # Higher angle indicates better dorsiflexion
            "back_angle": avg_back_angle,                # Should stay upright (close to 90°)
            "knee_alignment": 100 - avg_knee_alignment*100,  # Higher is better (knees tracking over toes)
            "deep_squat": deep_squat,
            "knees_past_toes": knees_past_toes,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    # Check squat depth
    if not deep_squat:
        feedback["feedback"].append("Try to squat deeper. Aim to get your thighs at least parallel to the ground for full muscle engagement.")
    
    # Check back position
    if avg_back_angle < 70:
        feedback["feedback"].append("Try to keep your back more upright. Your torso is leaning too far forward during the squat.")
    
    # Check knee alignment
    if avg_knee_alignment > 0.1:
        feedback["feedback"].append("Focus on keeping your knees aligned with your toes. Your knees are tracking too far inward or outward.")
    
    # Check for knees past toes
    if knees_past_toes:
        feedback["feedback"].append("Your knees are moving too far forward past your toes. Try to push your hips back more as you descend.")
    
    # Check ankle mobility
    if avg_ankle_angle < 70:
        feedback["feedback"].append("Work on ankle mobility. Limited ankle dorsiflexion is affecting your squat depth and form.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your goblet squat shows good depth, proper back position, and good knee alignment.")
        
    return feedback

# Example usage - Make sure you have the correct file path
video_file = "clips/GobletSquat3.MOV"  # Update this to the correct path
result = analyze_goblet_squat(video_file, "clips/analyzed_squat3.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Squat depth (knee angle, lower is deeper): {result['form_analysis']['squat_depth']:.1f}°")
    print(f"Hip mobility (hip angle, lower is better): {result['form_analysis']['hip_mobility']:.1f}°")
    print(f"Ankle mobility: {result['form_analysis']['ankle_mobility']:.1f}°")
    print(f"Back angle (90° is upright): {result['form_analysis']['back_angle']:.1f}°")
    print(f"Knee alignment score: {result['form_analysis']['knee_alignment']:.1f}%")
    print(f"Achieved full depth: {'Yes' if result['form_analysis']['deep_squat'] else 'No'}")
    print(f"Knees past toes: {'Yes' if result['form_analysis']['knees_past_toes'] else 'No'}")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745947640.092082 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745947640.213394 6332333 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745947640.228408 6332341 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 30, Total frames: 114
Detected 33 landmarks
Processing frame 100/114

=== ANALYSIS RESULTS ===
Counted 0 reps
Squat depth (knee angle, lower is deeper): 135.3°
Hip mobility (hip angle, lower is better): 126.4°
Ankle mobility: 163.1°
Back angle (90° is upright): 88.0°
Knee alignment score: 88.4%
Achieved full depth: No
Knees past toes: Yes
Frames analyzed: 114

Feedback:
- Try to squat deeper. Aim to get your thighs at least parallel to the ground for full muscle engagement.
- Focus on keeping your knees aligned with your toes. Your knees are tracking too far inward or outward.
- Your knees are moving too far forward past your toes. Try to push your hips back more as you descend.


In [8]:
import mediapipe as mp
import cv2
import numpy as np
import math
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_rdl(video_path, output_video_path=None):
    # Check if file exists
    if not os.path.isfile(video_path):
        print(f"Error: File '{video_path}' does not exist!")
        return {
            "rep_count": 0,
            "error": f"Video file not found: {video_path}"
        }
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return {
            "rep_count": 0,
            "error": f"Could not open video file: {video_path}. Check file format and codecs."
        }
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {total_frames}")
    
    # If we still have 0x0 dimensions, there's a codec issue
    if frame_width == 0 or frame_height == 0:
        print(f"Error: Could not determine video dimensions. Try converting the video to MP4 format.")
        return {
            "rep_count": 0,
            "error": "Could not determine video dimensions. Try converting to MP4 format."
        }
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or try 'avc1'
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track RDL state
    rep_count = 0
    rdl_stage = None  # "up" (standing) or "down" (hips back, torso forward)
    good_frames = 0
    
    # Lists to store angles and positions for analysis
    hip_angles = []       # Hip hinge angle
    knee_angles = []      # Knee angle (slight bend is good)
    back_angles = []      # Back angle relative to horizontal (should remain flat)
    torso_angles = []     # Torso angle relative to vertical
    shins_vertical = []   # Track if shins remain vertical
    bar_path = []         # Track bar path (approximated by wrist position)
    hip_depths = []       # Track how far back the hips travel
    
    frame_count = 0
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get key points for RDL analysis
            # We'll analyze both sides and average for better accuracy
            
            # Foot points
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            left_heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]
            left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_heel = [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y]
            right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
            
            # Leg points
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Upper body points
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            
            # Wrist points to approximate bar position
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate mid points for better analysis
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, 
                          (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, 
                      (left_hip[1] + right_hip[1])/2]
            mid_knee = [(left_knee[0] + right_knee[0])/2, 
                       (left_knee[1] + right_knee[1])/2]
            mid_ankle = [(left_ankle[0] + right_ankle[0])/2, 
                        (left_ankle[1] + right_ankle[1])/2]
            mid_wrist = [(left_wrist[0] + right_wrist[0])/2, 
                        (left_wrist[1] + right_wrist[1])/2]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Calculate knee angles (hip-knee-ankle)
            left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            avg_knee_angle = (left_knee_angle + right_knee_angle) / 2
            knee_angles.append(avg_knee_angle)
            
            # Calculate hip angle (shoulder-hip-knee) - this measures the hip hinge
            left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            avg_hip_angle = (left_hip_angle + right_hip_angle) / 2
            hip_angles.append(avg_hip_angle)
            
            # Calculate torso angle relative to vertical
            # Create a point directly below mid_hip to represent vertical
            vertical_point = [mid_hip[0], mid_hip[1] + 0.2]  # Adding to y moves down in image coords
            torso_angle = calculate_angle(mid_shoulder, mid_hip, vertical_point)
            torso_angle = 180 - torso_angle if torso_angle > 90 else torso_angle
            torso_angles.append(torso_angle)
            
            # Calculate back angle relative to horizontal
            # Create a point directly to the right of mid_hip to represent horizontal
            horizontal_point = [mid_hip[0] + 0.2, mid_hip[1]]
            back_angle = calculate_angle(mid_shoulder, mid_hip, horizontal_point)
            back_angles.append(back_angle)
            
            # Check if shins remain vertical
            # Calculate shin angle (knee-ankle-foot)
            left_shin_angle = calculate_angle(left_knee, left_ankle, left_foot_index)
            right_shin_angle = calculate_angle(right_knee, right_ankle, right_foot_index)
            # Adjust angles to measure deviation from vertical (0° is vertical)
            left_shin_vertical = abs(left_shin_angle - 90)
            right_shin_vertical = abs(right_shin_angle - 90)
            avg_shin_vertical = (left_shin_vertical + right_shin_vertical) / 2
            shins_vertical.append(avg_shin_vertical)
            
            # Track bar path (using wrist position as proxy)
            bar_path.append(mid_wrist)
            
            # Track hip depth (horizontal distance from hip to ankle)
            hip_depth = abs(mid_hip[0] - mid_ankle[0])
            hip_depths.append(hip_depth)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            hip_px = normalized_to_pixel_coordinates(mid_hip[0], mid_hip[1], frame_width, frame_height)
            knee_px = normalized_to_pixel_coordinates(mid_knee[0], mid_knee[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"Hip angle: {avg_hip_angle:.1f}°",
                        (hip_px[0] - 70, hip_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Knee angle: {avg_knee_angle:.1f}°",
                        (knee_px[0] - 70, knee_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Torso angle: {torso_angle:.1f}°",
                        (hip_px[0] - 70, hip_px[1] + 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine RDL stage based on hip angle and torso angle
            # For RDLs, "down" is when hips are pushed back and torso is forward
            # "up" is when standing upright
            
            # RDL is "down" when hip angle is smaller and torso is more horizontal
            if avg_hip_angle < 100 and torso_angle > 45 and (rdl_stage == "up" or rdl_stage is None):
                rdl_stage = "down"
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # RDL is "up" when hip angle is larger and torso is more vertical
            elif avg_hip_angle > 160 and torso_angle < 20 and rdl_stage == "down":
                rdl_stage = "up"
                # Coming from "down", count a rep
                rep_count += 1
                print(f"Rep #{rep_count} detected at frame with hip angle {avg_hip_angle:.1f}°")
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path and 'out' in locals():
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    min_hip_angle = np.min(hip_angles) if hip_angles else 180
    avg_knee_angle = np.mean(knee_angles) if knee_angles else 180
    avg_torso_angle = np.mean(torso_angles) if torso_angles else 0
    max_torso_angle = np.max(torso_angles) if torso_angles else 0
    
    # Calculate back flatness
    # Check if back angle changes significantly during the movement
    back_flatness = 100 - (np.std(back_angles) * 10) if back_angles else 0
    back_flatness = max(0, min(100, back_flatness))  # Clamp between 0-100
    
    # Calculate shin verticality (lower is better)
    shin_verticality = np.mean(shins_vertical) if shins_vertical else 45
    shin_verticality_score = max(0, 100 - (shin_verticality * 2))  # Convert to 0-100 score
    
    # Calculate bar path consistency
    # Measure horizontal deviation in bar path
    if len(bar_path) > 1:
        bar_x_positions = [pos[0] for pos in bar_path]
        bar_path_consistency = 100 - (np.std(bar_x_positions) * 500)  # Convert to 0-100 score
        bar_path_consistency = max(0, min(100, bar_path_consistency))  # Clamp between 0-100
    else:
        bar_path_consistency = 0
    
    # Calculate hip hinge quality
    max_hip_depth = np.max(hip_depths) if hip_depths else 0
    hip_hinge_quality = min(100, max_hip_depth * 200)  # Convert to 0-100 score
    
    # Check for key RDL form criteria
    hip_hinge_sufficient = min_hip_angle < 110  # Did they hinge enough at the hips?
    knees_slightly_bent = 140 < avg_knee_angle < 170  # Knees should be slightly bent
    back_flat = back_flatness > 70  # Back stayed relatively flat
    proper_shin_angle = shin_verticality_score > 70  # Shins stayed close to vertical
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "hip_hinge": min_hip_angle,              # Lower value means greater hip hinge
            "hip_hinge_quality": hip_hinge_quality,  # Higher is better (0-100)
            "knee_angle": avg_knee_angle,            # Should be slightly bent (140-170°)
            "back_flatness": back_flatness,          # Higher is better (0-100)
            "shin_verticality": shin_verticality_score,  # Higher is better (0-100)
            "bar_path": bar_path_consistency,        # Higher is better (0-100)
            "max_forward_lean": max_torso_angle,     # Higher means more forward lean
            "hip_hinge_sufficient": hip_hinge_sufficient,
            "knees_slightly_bent": knees_slightly_bent,
            "back_flat": back_flat,
            "proper_shin_angle": proper_shin_angle,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    # Check hip hinge
    if not hip_hinge_sufficient:
        feedback["feedback"].append("Focus on pushing your hips further back. Your hip hinge isn't deep enough for proper hamstring engagement.")
    
    # Check knee bend
    if avg_knee_angle > 170:
        feedback["feedback"].append("Keep a slight bend in your knees throughout the movement. Your knees are too straight.")
    elif avg_knee_angle < 140:
        feedback["feedback"].append("Your knees are bending too much. For an RDL, keep only a slight bend in the knees and focus on hinging at the hips.")
    
    # Check back position
    if not back_flat:
        feedback["feedback"].append("Focus on maintaining a flat back throughout the movement. Your back is rounding, which can increase injury risk.")
    
    # Check shin angle
    if not proper_shin_angle:
        feedback["feedback"].append("Try to keep your shins more vertical throughout the movement. Your shins are angling forward too much.")
    
    # Check bar path
    if bar_path_consistency < 70:
        feedback["feedback"].append("Work on keeping the bar path more consistent. The bar should travel in a straight vertical line close to your legs.")
    
    # Check forward lean
    if max_torso_angle < 30:
        feedback["feedback"].append("You're not leaning forward enough. In a proper RDL, your torso should reach a more horizontal position.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your RDL shows good hip hinge, proper knee bend, flat back, and a consistent bar path.")
        
    return feedback

# Example usage - Make sure you have the correct file path
video_file = "clips/RDL1.MOV"  # Update this to the correct path
result = analyze_rdl(video_file, "clips/analyzed_rdl1.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Hip hinge (lower means deeper hinge): {result['form_analysis']['hip_hinge']:.1f}°")
    print(f"Hip hinge quality score: {result['form_analysis']['hip_hinge_quality']:.1f}/100")
    print(f"Knee angle (should be 140-170°): {result['form_analysis']['knee_angle']:.1f}°")
    print(f"Back flatness score: {result['form_analysis']['back_flatness']:.1f}/100")
    print(f"Shin verticality score: {result['form_analysis']['shin_verticality']:.1f}/100")
    print(f"Bar path consistency score: {result['form_analysis']['bar_path']:.1f}/100")
    print(f"Maximum forward lean: {result['form_analysis']['max_forward_lean']:.1f}°")
    print(f"Hip hinge sufficient: {'Yes' if result['form_analysis']['hip_hinge_sufficient'] else 'No'}")
    print(f"Knees properly bent: {'Yes' if result['form_analysis']['knees_slightly_bent'] else 'No'}")
    print(f"Back remained flat: {'Yes' if result['form_analysis']['back_flat'] else 'No'}")
    print(f"Shins remained vertical: {'Yes' if result['form_analysis']['proper_shin_angle'] else 'No'}")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745947839.971401 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745947840.059931 6338055 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745947840.071497 6338053 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29, Total frames: 474
Detected 33 landmarks
Processing frame 100/474
Processing frame 200/474
Processing frame 300/474
Processing frame 400/474

=== ANALYSIS RESULTS ===
Counted 0 reps
Hip hinge (lower means deeper hinge): 56.8°
Hip hinge quality score: 45.1/100
Knee angle (should be 140-170°): 156.3°
Back flatness score: 0.0/100
Shin verticality score: 53.3/100
Bar path consistency score: 77.7/100
Maximum forward lean: 90.0°
Hip hinge sufficient: Yes
Knees properly bent: Yes
Back remained flat: No
Shins remained vertical: No
Frames analyzed: 474

Feedback:
- Focus on maintaining a flat back throughout the movement. Your back is rounding, which can increase injury risk.
- Try to keep your shins more vertical throughout the movement. Your shins are angling forward too much.


In [9]:
import mediapipe as mp
import cv2
import numpy as np
import math
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_rdl(video_path, output_video_path=None):
    # Check if file exists
    if not os.path.isfile(video_path):
        print(f"Error: File '{video_path}' does not exist!")
        return {
            "rep_count": 0,
            "error": f"Video file not found: {video_path}"
        }
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return {
            "rep_count": 0,
            "error": f"Could not open video file: {video_path}. Check file format and codecs."
        }
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {total_frames}")
    
    # If we still have 0x0 dimensions, there's a codec issue
    if frame_width == 0 or frame_height == 0:
        print(f"Error: Could not determine video dimensions. Try converting the video to MP4 format.")
        return {
            "rep_count": 0,
            "error": "Could not determine video dimensions. Try converting to MP4 format."
        }
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or try 'avc1'
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track RDL state
    rep_count = 0
    rdl_stage = None  # "up" (standing) or "down" (hips back, torso forward)
    good_frames = 0
    
    # Lists to store angles and positions for analysis
    hip_angles = []       # Hip hinge angle
    knee_angles = []      # Knee angle (slight bend is good)
    back_angles = []      # Back angle relative to horizontal (should remain flat)
    torso_angles = []     # Torso angle relative to vertical
    shins_vertical = []   # Track if shins remain vertical
    bar_path = []         # Track bar path (approximated by wrist position)
    hip_depths = []       # Track how far back the hips travel
    
    frame_count = 0
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get key points for RDL analysis
            # We'll analyze both sides and average for better accuracy
            
            # Foot points
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            left_heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]
            left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_heel = [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y]
            right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
            
            # Leg points
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Upper body points
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            
            # Wrist points to approximate bar position
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate mid points for better analysis
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, 
                          (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, 
                      (left_hip[1] + right_hip[1])/2]
            mid_knee = [(left_knee[0] + right_knee[0])/2, 
                       (left_knee[1] + right_knee[1])/2]
            mid_ankle = [(left_ankle[0] + right_ankle[0])/2, 
                        (left_ankle[1] + right_ankle[1])/2]
            mid_wrist = [(left_wrist[0] + right_wrist[0])/2, 
                        (left_wrist[1] + right_wrist[1])/2]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Calculate knee angles (hip-knee-ankle)
            left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            avg_knee_angle = (left_knee_angle + right_knee_angle) / 2
            knee_angles.append(avg_knee_angle)
            
            # Calculate hip angle (shoulder-hip-knee) - this measures the hip hinge
            left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            avg_hip_angle = (left_hip_angle + right_hip_angle) / 2
            hip_angles.append(avg_hip_angle)
            
            # Calculate torso angle relative to vertical
            # Create a point directly below mid_hip to represent vertical
            vertical_point = [mid_hip[0], mid_hip[1] + 0.2]  # Adding to y moves down in image coords
            torso_angle = calculate_angle(mid_shoulder, mid_hip, vertical_point)
            torso_angle = 180 - torso_angle if torso_angle > 90 else torso_angle
            torso_angles.append(torso_angle)
            
            # Calculate back angle relative to horizontal
            # Create a point directly to the right of mid_hip to represent horizontal
            horizontal_point = [mid_hip[0] + 0.2, mid_hip[1]]
            back_angle = calculate_angle(mid_shoulder, mid_hip, horizontal_point)
            back_angles.append(back_angle)
            
            # Check if shins remain vertical
            # Calculate shin angle (knee-ankle-foot)
            left_shin_angle = calculate_angle(left_knee, left_ankle, left_foot_index)
            right_shin_angle = calculate_angle(right_knee, right_ankle, right_foot_index)
            # Adjust angles to measure deviation from vertical (0° is vertical)
            left_shin_vertical = abs(left_shin_angle - 90)
            right_shin_vertical = abs(right_shin_angle - 90)
            avg_shin_vertical = (left_shin_vertical + right_shin_vertical) / 2
            shins_vertical.append(avg_shin_vertical)
            
            # Track bar path (using wrist position as proxy)
            bar_path.append(mid_wrist)
            
            # Track hip depth (horizontal distance from hip to ankle)
            hip_depth = abs(mid_hip[0] - mid_ankle[0])
            hip_depths.append(hip_depth)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            hip_px = normalized_to_pixel_coordinates(mid_hip[0], mid_hip[1], frame_width, frame_height)
            knee_px = normalized_to_pixel_coordinates(mid_knee[0], mid_knee[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"Hip angle: {avg_hip_angle:.1f}°",
                        (hip_px[0] - 70, hip_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Knee angle: {avg_knee_angle:.1f}°",
                        (knee_px[0] - 70, knee_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Torso angle: {torso_angle:.1f}°",
                        (hip_px[0] - 70, hip_px[1] + 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine RDL stage based on hip angle and torso angle
            # For RDLs, "down" is when hips are pushed back and torso is forward
            # "up" is when standing upright
            
            # RDL is "down" when hip angle is smaller and torso is more horizontal
            if avg_hip_angle < 100 and torso_angle > 45 and (rdl_stage == "up" or rdl_stage is None):
                rdl_stage = "down"
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # RDL is "up" when hip angle is larger and torso is more vertical
            elif avg_hip_angle > 160 and torso_angle < 20 and rdl_stage == "down":
                rdl_stage = "up"
                # Coming from "down", count a rep
                rep_count += 1
                print(f"Rep #{rep_count} detected at frame with hip angle {avg_hip_angle:.1f}°")
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path and 'out' in locals():
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    min_hip_angle = np.min(hip_angles) if hip_angles else 180
    avg_knee_angle = np.mean(knee_angles) if knee_angles else 180
    avg_torso_angle = np.mean(torso_angles) if torso_angles else 0
    max_torso_angle = np.max(torso_angles) if torso_angles else 0
    
    # Calculate back flatness
    # Check if back angle changes significantly during the movement
    back_flatness = 100 - (np.std(back_angles) * 10) if back_angles else 0
    back_flatness = max(0, min(100, back_flatness))  # Clamp between 0-100
    
    # Calculate shin verticality (lower is better)
    shin_verticality = np.mean(shins_vertical) if shins_vertical else 45
    shin_verticality_score = max(0, 100 - (shin_verticality * 2))  # Convert to 0-100 score
    
    # Calculate bar path consistency
    # Measure horizontal deviation in bar path
    if len(bar_path) > 1:
        bar_x_positions = [pos[0] for pos in bar_path]
        bar_path_consistency = 100 - (np.std(bar_x_positions) * 500)  # Convert to 0-100 score
        bar_path_consistency = max(0, min(100, bar_path_consistency))  # Clamp between 0-100
    else:
        bar_path_consistency = 0
    
    # Calculate hip hinge quality
    max_hip_depth = np.max(hip_depths) if hip_depths else 0
    hip_hinge_quality = min(100, max_hip_depth * 200)  # Convert to 0-100 score
    
    # Check for key RDL form criteria
    hip_hinge_sufficient = min_hip_angle < 110  # Did they hinge enough at the hips?
    knees_slightly_bent = 140 < avg_knee_angle < 170  # Knees should be slightly bent
    back_flat = back_flatness > 70  # Back stayed relatively flat
    proper_shin_angle = shin_verticality_score > 70  # Shins stayed close to vertical
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "hip_hinge": min_hip_angle,              # Lower value means greater hip hinge
            "hip_hinge_quality": hip_hinge_quality,  # Higher is better (0-100)
            "knee_angle": avg_knee_angle,            # Should be slightly bent (140-170°)
            "back_flatness": back_flatness,          # Higher is better (0-100)
            "shin_verticality": shin_verticality_score,  # Higher is better (0-100)
            "bar_path": bar_path_consistency,        # Higher is better (0-100)
            "max_forward_lean": max_torso_angle,     # Higher means more forward lean
            "hip_hinge_sufficient": hip_hinge_sufficient,
            "knees_slightly_bent": knees_slightly_bent,
            "back_flat": back_flat,
            "proper_shin_angle": proper_shin_angle,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    # Check hip hinge
    if not hip_hinge_sufficient:
        feedback["feedback"].append("Focus on pushing your hips further back. Your hip hinge isn't deep enough for proper hamstring engagement.")
    
    # Check knee bend
    if avg_knee_angle > 170:
        feedback["feedback"].append("Keep a slight bend in your knees throughout the movement. Your knees are too straight.")
    elif avg_knee_angle < 140:
        feedback["feedback"].append("Your knees are bending too much. For an RDL, keep only a slight bend in the knees and focus on hinging at the hips.")
    
    # Check back position
    if not back_flat:
        feedback["feedback"].append("Focus on maintaining a flat back throughout the movement. Your back is rounding, which can increase injury risk.")
    
    # Check shin angle
    if not proper_shin_angle:
        feedback["feedback"].append("Try to keep your shins more vertical throughout the movement. Your shins are angling forward too much.")
    
    # Check bar path
    if bar_path_consistency < 70:
        feedback["feedback"].append("Work on keeping the bar path more consistent. The bar should travel in a straight vertical line close to your legs.")
    
    # Check forward lean
    if max_torso_angle < 30:
        feedback["feedback"].append("You're not leaning forward enough. In a proper RDL, your torso should reach a more horizontal position.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your RDL shows good hip hinge, proper knee bend, flat back, and a consistent bar path.")
        
    return feedback

# Example usage - Make sure you have the correct file path
video_file = "clips/RDL2.MOV"  # Update this to the correct path
result = analyze_rdl(video_file, "clips/analyzed_rdl2.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Hip hinge (lower means deeper hinge): {result['form_analysis']['hip_hinge']:.1f}°")
    print(f"Hip hinge quality score: {result['form_analysis']['hip_hinge_quality']:.1f}/100")
    print(f"Knee angle (should be 140-170°): {result['form_analysis']['knee_angle']:.1f}°")
    print(f"Back flatness score: {result['form_analysis']['back_flatness']:.1f}/100")
    print(f"Shin verticality score: {result['form_analysis']['shin_verticality']:.1f}/100")
    print(f"Bar path consistency score: {result['form_analysis']['bar_path']:.1f}/100")
    print(f"Maximum forward lean: {result['form_analysis']['max_forward_lean']:.1f}°")
    print(f"Hip hinge sufficient: {'Yes' if result['form_analysis']['hip_hinge_sufficient'] else 'No'}")
    print(f"Knees properly bent: {'Yes' if result['form_analysis']['knees_slightly_bent'] else 'No'}")
    print(f"Back remained flat: {'Yes' if result['form_analysis']['back_flat'] else 'No'}")
    print(f"Shins remained vertical: {'Yes' if result['form_analysis']['proper_shin_angle'] else 'No'}")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745947914.124683 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745947914.242110 6340080 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745947914.256765 6340080 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29, Total frames: 599
Detected 33 landmarks
Processing frame 100/599
Processing frame 200/599
Processing frame 300/599
Processing frame 400/599
Processing frame 500/599

=== ANALYSIS RESULTS ===
Counted 0 reps
Hip hinge (lower means deeper hinge): 38.5°
Hip hinge quality score: 58.1/100
Knee angle (should be 140-170°): 140.0°
Back flatness score: 0.0/100
Shin verticality score: 82.0/100
Bar path consistency score: 63.3/100
Maximum forward lean: 90.0°
Hip hinge sufficient: Yes
Knees properly bent: No
Back remained flat: No
Shins remained vertical: Yes
Frames analyzed: 599

Feedback:
- Your knees are bending too much. For an RDL, keep only a slight bend in the knees and focus on hinging at the hips.
- Focus on maintaining a flat back throughout the movement. Your back is rounding, which can increase injury risk.
- Work on keeping the bar path more consistent. The bar should travel in a straight vertical line close to your legs.


In [10]:
import mediapipe as mp
import cv2
import numpy as np
import math
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_reverse_lunge(video_path, output_video_path=None):
    # Check if file exists
    if not os.path.isfile(video_path):
        print(f"Error: File '{video_path}' does not exist!")
        return {
            "rep_count": 0,
            "error": f"Video file not found: {video_path}"
        }
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return {
            "rep_count": 0,
            "error": f"Could not open video file: {video_path}. Check file format and codecs."
        }
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {total_frames}")
    
    # If we still have 0x0 dimensions, there's a codec issue
    if frame_width == 0 or frame_height == 0:
        print(f"Error: Could not determine video dimensions. Try converting the video to MP4 format.")
        return {
            "rep_count": 0,
            "error": "Could not determine video dimensions. Try converting to MP4 format."
        }
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or try 'avc1'
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track lunge state
    rep_count = 0
    lunge_stage = None  # "up" (standing) or "down" (in lunge position)
    good_frames = 0
    
    # Determine which leg is stepping back by tracking movements
    active_leg = None  # "left" or "right"
    left_ankle_positions = []
    right_ankle_positions = []
    
    # Lists to store angles and positions for analysis
    front_knee_angles = []  # Front knee should be ~90 degrees at bottom
    back_knee_angles = []   # Back knee should almost touch the ground
    torso_angles = []       # Torso should stay upright
    stride_lengths = []     # Distance between feet
    knee_alignments = []    # Front knee should stay aligned with ankle
    hip_heights = []        # Hip drop/rotation
    
    frame_count = 0
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get all key points for lunge analysis
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]
            left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_heel = [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y]
            right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
            
            # Calculate mid points
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, 
                          (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, 
                      (left_hip[1] + right_hip[1])/2]
            
            # Track ankle positions to determine which leg steps back
            left_ankle_positions.append(left_ankle)
            right_ankle_positions.append(right_ankle)
            
            # If we have enough frames, determine which leg is moving back (the active lunge leg)
            if good_frames == 30 and active_leg is None:
                # Calculate movement for each ankle
                left_movement = 0
                right_movement = 0
                
                for i in range(1, len(left_ankle_positions)):
                    left_movement += abs(left_ankle_positions[i][0] - left_ankle_positions[i-1][0])
                    left_movement += abs(left_ankle_positions[i][1] - left_ankle_positions[i-1][1])
                    
                    right_movement += abs(right_ankle_positions[i][0] - right_ankle_positions[i-1][0])
                    right_movement += abs(right_ankle_positions[i][1] - right_ankle_positions[i-1][1])
                
                # The leg with more movement is likely the one stepping back
                if left_movement > right_movement:
                    active_leg = "left"
                else:
                    active_leg = "right"
                    
                print(f"Detected {active_leg} leg as the stepping leg for reverse lunges")
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Determine front and back legs based on detected active leg
            # If left leg is stepping back, right leg is the front
            if active_leg == "left" or active_leg is None:
                front_ankle = right_ankle
                front_knee = right_knee
                front_hip = right_hip
                front_foot_index = right_foot_index
                
                back_ankle = left_ankle
                back_knee = left_knee
                back_hip = left_hip
                back_foot_index = left_foot_index
                
                front_label = "R"
                back_label = "L"
            else:
                front_ankle = left_ankle
                front_knee = left_knee
                front_hip = left_hip
                front_foot_index = left_foot_index
                
                back_ankle = right_ankle
                back_knee = right_knee
                back_hip = right_hip
                back_foot_index = right_foot_index
                
                front_label = "L"
                back_label = "R"
            
            # Calculate front knee angle (hip-knee-ankle)
            front_knee_angle = calculate_angle(front_hip, front_knee, front_ankle)
            front_knee_angles.append(front_knee_angle)
            
            # Calculate back knee angle (hip-knee-ankle)
            back_knee_angle = calculate_angle(back_hip, back_knee, back_ankle)
            back_knee_angles.append(back_knee_angle)
            
            # Calculate torso angle relative to vertical
            # Create a point directly below mid_hip to represent vertical
            vertical_point = [mid_hip[0], mid_hip[1] + 0.2]  # Adding to y moves down in image coords
            torso_angle = calculate_angle(mid_shoulder, mid_hip, vertical_point)
            torso_angle = 180 - torso_angle if torso_angle > 90 else torso_angle
            torso_angles.append(torso_angle)
            
            # Calculate stride length (distance between ankles)
            stride_length = np.sqrt((front_ankle[0] - back_ankle[0])**2 + 
                                  (front_ankle[1] - back_ankle[1])**2)
            stride_lengths.append(stride_length)
            
            # Calculate front knee alignment
            # Measure horizontal distance from knee to ankle to track knee position
            front_knee_to_ankle_x = front_knee[0] - front_ankle[0]
            knee_alignments.append(front_knee_to_ankle_x)
            
            # Calculate hip drop/rotation
            # Measure height difference between hips
            hip_height_diff = abs(left_hip[1] - right_hip[1])
            hip_heights.append(hip_height_diff)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            front_knee_px = normalized_to_pixel_coordinates(front_knee[0], front_knee[1], frame_width, frame_height)
            back_knee_px = normalized_to_pixel_coordinates(back_knee[0], back_knee[1], frame_width, frame_height)
            mid_hip_px = normalized_to_pixel_coordinates(mid_hip[0], mid_hip[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"{front_label} Knee: {front_knee_angle:.1f}°",
                        (front_knee_px[0] - 70, front_knee_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"{back_label} Knee: {back_knee_angle:.1f}°",
                        (back_knee_px[0] - 70, back_knee_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Torso: {torso_angle:.1f}°",
                        (mid_hip_px[0] - 50, mid_hip_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine lunge stage based on front knee angle
            # For lunges, "down" is when front knee is bent (smaller angle)
            # "up" is when standing upright (larger angle)
            
            # Lunge is "down" when front knee angle is small (in deep lunge position)
            if front_knee_angle < 120 and (lunge_stage == "up" or lunge_stage is None):
                lunge_stage = "down"
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # Lunge is "up" when front knee angle is large (standing position)
            elif front_knee_angle > 160 and lunge_stage == "down":
                lunge_stage = "up"
                # Coming from "down", count a rep
                rep_count += 1
                print(f"Rep #{rep_count} detected at frame with front knee angle {front_knee_angle:.1f}°")
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path and 'out' in locals():
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    min_front_knee_angle = np.min(front_knee_angles) if front_knee_angles else 180
    min_back_knee_angle = np.min(back_knee_angles) if back_knee_angles else 180
    avg_torso_angle = np.mean(torso_angles) if torso_angles else 0
    max_stride_length = np.max(stride_lengths) if stride_lengths else 0
    
    # Calculate front knee alignment (negative = knee behind ankle, positive = knee over ankle)
    knee_alignment_avg = np.mean(knee_alignments) if knee_alignments else 0
    knee_alignment_score = 100 - (abs(knee_alignment_avg) * 300)  # Convert to 0-100 score
    knee_alignment_score = max(0, min(100, knee_alignment_score))  # Clamp between 0-100
    
    # Calculate hip stability (lower is better)
    hip_stability = np.mean(hip_heights) if hip_heights else 0.1
    hip_stability_score = 100 - (hip_stability * 500)  # Convert to 0-100 score
    hip_stability_score = max(0, min(100, hip_stability_score))  # Clamp between 0-100
    
    # Check for key lunge form criteria
    front_knee_proper = 80 <= min_front_knee_angle <= 110  # ~90° is ideal
    back_knee_proper = min_back_knee_angle < 110  # Should bend significantly
    torso_upright = avg_torso_angle < 20  # Torso should stay relatively vertical
    knee_aligned = knee_alignment_score > 70  # Knee should track over ankle
    good_hip_stability = hip_stability_score > 70  # Hips should stay level
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "front_knee_angle": min_front_knee_angle,  # Should be ~90° at bottom
            "back_knee_angle": min_back_knee_angle,    # Should be bent significantly
            "torso_angle": avg_torso_angle,            # Should stay upright (<20°)
            "stride_length": max_stride_length,        # Higher is generally better for proper depth
            "knee_alignment": knee_alignment_score,    # Higher is better (0-100)
            "hip_stability": hip_stability_score,      # Higher is better (0-100)
            "front_knee_proper": front_knee_proper,
            "back_knee_proper": back_knee_proper,
            "torso_upright": torso_upright,
            "knee_aligned": knee_aligned,
            "good_hip_stability": good_hip_stability,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    # Check front knee angle
    if min_front_knee_angle > 110:
        feedback["feedback"].append("You're not going deep enough in your lunge. Aim for your front knee to bend to about 90 degrees.")
    elif min_front_knee_angle < 80:
        feedback["feedback"].append("Your front knee is bending too much (below 80°). This puts excessive pressure on the knee joint.")
    
    # Check back knee angle
    if min_back_knee_angle > 110:
        feedback["feedback"].append("Your back leg isn't bending enough. Allow your back knee to bend more as you lower into the lunge.")
    
    # Check torso position
    if avg_torso_angle > 20:
        feedback["feedback"].append("Try to keep your torso more upright. You're leaning forward too much during the lunge.")
    
    # Check knee alignment
    if not knee_aligned:
        if knee_alignment_avg > 0:
            feedback["feedback"].append("Your front knee is tracking too far forward over your toes. Keep your knee aligned with your ankle.")
        else:
            feedback["feedback"].append("Your front knee is positioned too far back. Ensure it's aligned properly over your ankle.")
    
    # Check hip stability
    if not good_hip_stability:
        feedback["feedback"].append("Focus on keeping your hips level throughout the movement. Your hips are rotating or dipping to one side.")
    
    # Check stride length
    if max_stride_length < 0.3:  # This threshold might need tuning
        feedback["feedback"].append("Your stride length is too short. Step back further to achieve proper lunge depth and form.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your reverse lunges show good depth, proper knee position, upright torso, and stable hips.")
        
    return feedback

# Example usage - Make sure you have the correct file path
video_file = "clips/ReverseLunge1.MOV"  # Update this to the correct path
result = analyze_reverse_lunge(video_file, "clips/analyzed_lunge1.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Front knee angle (ideal ~90°): {result['form_analysis']['front_knee_angle']:.1f}°")
    print(f"Back knee angle (lower is better): {result['form_analysis']['back_knee_angle']:.1f}°")
    print(f"Torso angle (lower is better): {result['form_analysis']['torso_angle']:.1f}°")
    print(f"Stride length: {result['form_analysis']['stride_length']:.3f}")
    print(f"Knee alignment score: {result['form_analysis']['knee_alignment']:.1f}/100")
    print(f"Hip stability score: {result['form_analysis']['hip_stability']:.1f}/100")
    print(f"Front knee proper position: {'Yes' if result['form_analysis']['front_knee_proper'] else 'No'}")
    print(f"Back knee proper bend: {'Yes' if result['form_analysis']['back_knee_proper'] else 'No'}")
    print(f"Torso stayed upright: {'Yes' if result['form_analysis']['torso_upright'] else 'No'}")
    print(f"Knee properly aligned: {'Yes' if result['form_analysis']['knee_aligned'] else 'No'}")
    print(f"Good hip stability: {'Yes' if result['form_analysis']['good_hip_stability'] else 'No'}")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745948138.650268 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745948138.739511 6346551 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745948138.758923 6346550 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29, Total frames: 468
Detected 33 landmarks
Detected left leg as the stepping leg for reverse lunges
Processing frame 100/468
Rep #1 detected at frame with front knee angle 164.1°
Processing frame 200/468
Rep #2 detected at frame with front knee angle 166.4°
Processing frame 300/468
Rep #3 detected at frame with front knee angle 172.1°
Rep #4 detected at frame with front knee angle 161.5°
Processing frame 400/468

=== ANALYSIS RESULTS ===
Counted 4 reps
Front knee angle (ideal ~90°): 21.4°
Back knee angle (lower is better): 0.3°
Torso angle (lower is better): 85.9°
Stride length: 0.122
Knee alignment score: 80.9/100
Hip stability score: 53.9/100
Front knee proper position: No
Back knee proper bend: Yes
Torso stayed upright: No
Knee properly aligned: Yes
Good hip stability: No
Frames analyzed: 468

Feedback:
- Your front knee is bending too much (below 80°). This puts excessive pressure on the knee joint.
- Try to keep your torso more upright. You're le

In [11]:
import mediapipe as mp
import cv2
import numpy as np
import math
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_reverse_lunge(video_path, output_video_path=None):
    # Check if file exists
    if not os.path.isfile(video_path):
        print(f"Error: File '{video_path}' does not exist!")
        return {
            "rep_count": 0,
            "error": f"Video file not found: {video_path}"
        }
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return {
            "rep_count": 0,
            "error": f"Could not open video file: {video_path}. Check file format and codecs."
        }
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {total_frames}")
    
    # If we still have 0x0 dimensions, there's a codec issue
    if frame_width == 0 or frame_height == 0:
        print(f"Error: Could not determine video dimensions. Try converting the video to MP4 format.")
        return {
            "rep_count": 0,
            "error": "Could not determine video dimensions. Try converting to MP4 format."
        }
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or try 'avc1'
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track lunge state
    rep_count = 0
    lunge_stage = None  # "up" (standing) or "down" (in lunge position)
    good_frames = 0
    
    # Determine which leg is stepping back by tracking movements
    active_leg = None  # "left" or "right"
    left_ankle_positions = []
    right_ankle_positions = []
    
    # Lists to store angles and positions for analysis
    front_knee_angles = []  # Front knee should be ~90 degrees at bottom
    back_knee_angles = []   # Back knee should almost touch the ground
    torso_angles = []       # Torso should stay upright
    stride_lengths = []     # Distance between feet
    knee_alignments = []    # Front knee should stay aligned with ankle
    hip_heights = []        # Hip drop/rotation
    
    frame_count = 0
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get all key points for lunge analysis
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]
            left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_heel = [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y]
            right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
            
            # Calculate mid points
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, 
                          (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, 
                      (left_hip[1] + right_hip[1])/2]
            
            # Track ankle positions to determine which leg steps back
            left_ankle_positions.append(left_ankle)
            right_ankle_positions.append(right_ankle)
            
            # If we have enough frames, determine which leg is moving back (the active lunge leg)
            if good_frames == 30 and active_leg is None:
                # Calculate movement for each ankle
                left_movement = 0
                right_movement = 0
                
                for i in range(1, len(left_ankle_positions)):
                    left_movement += abs(left_ankle_positions[i][0] - left_ankle_positions[i-1][0])
                    left_movement += abs(left_ankle_positions[i][1] - left_ankle_positions[i-1][1])
                    
                    right_movement += abs(right_ankle_positions[i][0] - right_ankle_positions[i-1][0])
                    right_movement += abs(right_ankle_positions[i][1] - right_ankle_positions[i-1][1])
                
                # The leg with more movement is likely the one stepping back
                if left_movement > right_movement:
                    active_leg = "left"
                else:
                    active_leg = "right"
                    
                print(f"Detected {active_leg} leg as the stepping leg for reverse lunges")
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Determine front and back legs based on detected active leg
            # If left leg is stepping back, right leg is the front
            if active_leg == "left" or active_leg is None:
                front_ankle = right_ankle
                front_knee = right_knee
                front_hip = right_hip
                front_foot_index = right_foot_index
                
                back_ankle = left_ankle
                back_knee = left_knee
                back_hip = left_hip
                back_foot_index = left_foot_index
                
                front_label = "R"
                back_label = "L"
            else:
                front_ankle = left_ankle
                front_knee = left_knee
                front_hip = left_hip
                front_foot_index = left_foot_index
                
                back_ankle = right_ankle
                back_knee = right_knee
                back_hip = right_hip
                back_foot_index = right_foot_index
                
                front_label = "L"
                back_label = "R"
            
            # Calculate front knee angle (hip-knee-ankle)
            front_knee_angle = calculate_angle(front_hip, front_knee, front_ankle)
            front_knee_angles.append(front_knee_angle)
            
            # Calculate back knee angle (hip-knee-ankle)
            back_knee_angle = calculate_angle(back_hip, back_knee, back_ankle)
            back_knee_angles.append(back_knee_angle)
            
            # Calculate torso angle relative to vertical
            # Create a point directly below mid_hip to represent vertical
            vertical_point = [mid_hip[0], mid_hip[1] + 0.2]  # Adding to y moves down in image coords
            torso_angle = calculate_angle(mid_shoulder, mid_hip, vertical_point)
            torso_angle = 180 - torso_angle if torso_angle > 90 else torso_angle
            torso_angles.append(torso_angle)
            
            # Calculate stride length (distance between ankles)
            stride_length = np.sqrt((front_ankle[0] - back_ankle[0])**2 + 
                                  (front_ankle[1] - back_ankle[1])**2)
            stride_lengths.append(stride_length)
            
            # Calculate front knee alignment
            # Measure horizontal distance from knee to ankle to track knee position
            front_knee_to_ankle_x = front_knee[0] - front_ankle[0]
            knee_alignments.append(front_knee_to_ankle_x)
            
            # Calculate hip drop/rotation
            # Measure height difference between hips
            hip_height_diff = abs(left_hip[1] - right_hip[1])
            hip_heights.append(hip_height_diff)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            front_knee_px = normalized_to_pixel_coordinates(front_knee[0], front_knee[1], frame_width, frame_height)
            back_knee_px = normalized_to_pixel_coordinates(back_knee[0], back_knee[1], frame_width, frame_height)
            mid_hip_px = normalized_to_pixel_coordinates(mid_hip[0], mid_hip[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"{front_label} Knee: {front_knee_angle:.1f}°",
                        (front_knee_px[0] - 70, front_knee_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"{back_label} Knee: {back_knee_angle:.1f}°",
                        (back_knee_px[0] - 70, back_knee_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Torso: {torso_angle:.1f}°",
                        (mid_hip_px[0] - 50, mid_hip_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine lunge stage based on front knee angle
            # For lunges, "down" is when front knee is bent (smaller angle)
            # "up" is when standing upright (larger angle)
            
            # Lunge is "down" when front knee angle is small (in deep lunge position)
            if front_knee_angle < 120 and (lunge_stage == "up" or lunge_stage is None):
                lunge_stage = "down"
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # Lunge is "up" when front knee angle is large (standing position)
            elif front_knee_angle > 160 and lunge_stage == "down":
                lunge_stage = "up"
                # Coming from "down", count a rep
                rep_count += 1
                print(f"Rep #{rep_count} detected at frame with front knee angle {front_knee_angle:.1f}°")
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path and 'out' in locals():
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    min_front_knee_angle = np.min(front_knee_angles) if front_knee_angles else 180
    min_back_knee_angle = np.min(back_knee_angles) if back_knee_angles else 180
    avg_torso_angle = np.mean(torso_angles) if torso_angles else 0
    max_stride_length = np.max(stride_lengths) if stride_lengths else 0
    
    # Calculate front knee alignment (negative = knee behind ankle, positive = knee over ankle)
    knee_alignment_avg = np.mean(knee_alignments) if knee_alignments else 0
    knee_alignment_score = 100 - (abs(knee_alignment_avg) * 300)  # Convert to 0-100 score
    knee_alignment_score = max(0, min(100, knee_alignment_score))  # Clamp between 0-100
    
    # Calculate hip stability (lower is better)
    hip_stability = np.mean(hip_heights) if hip_heights else 0.1
    hip_stability_score = 100 - (hip_stability * 500)  # Convert to 0-100 score
    hip_stability_score = max(0, min(100, hip_stability_score))  # Clamp between 0-100
    
    # Check for key lunge form criteria
    front_knee_proper = 80 <= min_front_knee_angle <= 110  # ~90° is ideal
    back_knee_proper = min_back_knee_angle < 110  # Should bend significantly
    torso_upright = avg_torso_angle < 20  # Torso should stay relatively vertical
    knee_aligned = knee_alignment_score > 70  # Knee should track over ankle
    good_hip_stability = hip_stability_score > 70  # Hips should stay level
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "front_knee_angle": min_front_knee_angle,  # Should be ~90° at bottom
            "back_knee_angle": min_back_knee_angle,    # Should be bent significantly
            "torso_angle": avg_torso_angle,            # Should stay upright (<20°)
            "stride_length": max_stride_length,        # Higher is generally better for proper depth
            "knee_alignment": knee_alignment_score,    # Higher is better (0-100)
            "hip_stability": hip_stability_score,      # Higher is better (0-100)
            "front_knee_proper": front_knee_proper,
            "back_knee_proper": back_knee_proper,
            "torso_upright": torso_upright,
            "knee_aligned": knee_aligned,
            "good_hip_stability": good_hip_stability,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    # Check front knee angle
    if min_front_knee_angle > 110:
        feedback["feedback"].append("You're not going deep enough in your lunge. Aim for your front knee to bend to about 90 degrees.")
    elif min_front_knee_angle < 80:
        feedback["feedback"].append("Your front knee is bending too much (below 80°). This puts excessive pressure on the knee joint.")
    
    # Check back knee angle
    if min_back_knee_angle > 110:
        feedback["feedback"].append("Your back leg isn't bending enough. Allow your back knee to bend more as you lower into the lunge.")
    
    # Check torso position
    if avg_torso_angle > 20:
        feedback["feedback"].append("Try to keep your torso more upright. You're leaning forward too much during the lunge.")
    
    # Check knee alignment
    if not knee_aligned:
        if knee_alignment_avg > 0:
            feedback["feedback"].append("Your front knee is tracking too far forward over your toes. Keep your knee aligned with your ankle.")
        else:
            feedback["feedback"].append("Your front knee is positioned too far back. Ensure it's aligned properly over your ankle.")
    
    # Check hip stability
    if not good_hip_stability:
        feedback["feedback"].append("Focus on keeping your hips level throughout the movement. Your hips are rotating or dipping to one side.")
    
    # Check stride length
    if max_stride_length < 0.3:  # This threshold might need tuning
        feedback["feedback"].append("Your stride length is too short. Step back further to achieve proper lunge depth and form.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your reverse lunges show good depth, proper knee position, upright torso, and stable hips.")
        
    return feedback

# Example usage - Make sure you have the correct file path
video_file = "clips/ReverseLunge2.MOV"  # Update this to the correct path
result = analyze_reverse_lunge(video_file, "clips/analyzed_lunge2.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Front knee angle (ideal ~90°): {result['form_analysis']['front_knee_angle']:.1f}°")
    print(f"Back knee angle (lower is better): {result['form_analysis']['back_knee_angle']:.1f}°")
    print(f"Torso angle (lower is better): {result['form_analysis']['torso_angle']:.1f}°")
    print(f"Stride length: {result['form_analysis']['stride_length']:.3f}")
    print(f"Knee alignment score: {result['form_analysis']['knee_alignment']:.1f}/100")
    print(f"Hip stability score: {result['form_analysis']['hip_stability']:.1f}/100")
    print(f"Front knee proper position: {'Yes' if result['form_analysis']['front_knee_proper'] else 'No'}")
    print(f"Back knee proper bend: {'Yes' if result['form_analysis']['back_knee_proper'] else 'No'}")
    print(f"Torso stayed upright: {'Yes' if result['form_analysis']['torso_upright'] else 'No'}")
    print(f"Knee properly aligned: {'Yes' if result['form_analysis']['knee_aligned'] else 'No'}")
    print(f"Good hip stability: {'Yes' if result['form_analysis']['good_hip_stability'] else 'No'}")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745948174.471411 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745948174.551790 6347512 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745948174.564542 6347514 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29, Total frames: 679
Detected 33 landmarks
Detected right leg as the stepping leg for reverse lunges
Processing frame 100/679
Rep #1 detected at frame with front knee angle 167.8°
Rep #2 detected at frame with front knee angle 161.8°
Processing frame 200/679
Processing frame 300/679
Processing frame 400/679
Rep #3 detected at frame with front knee angle 163.8°
Processing frame 500/679
Processing frame 600/679

=== ANALYSIS RESULTS ===
Counted 3 reps
Front knee angle (ideal ~90°): 70.1°
Back knee angle (lower is better): 52.4°
Torso angle (lower is better): 85.8°
Stride length: 0.329
Knee alignment score: 66.0/100
Hip stability score: 11.5/100
Front knee proper position: No
Back knee proper bend: Yes
Torso stayed upright: No
Knee properly aligned: No
Good hip stability: No
Frames analyzed: 679

Feedback:
- Your front knee is bending too much (below 80°). This puts excessive pressure on the knee joint.
- Try to keep your torso more upright. You're leani

In [12]:
import mediapipe as mp
import cv2
import numpy as np
import math
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_shoulder_press(video_path, output_video_path=None):
    # Check if file exists
    if not os.path.isfile(video_path):
        print(f"Error: File '{video_path}' does not exist!")
        return {
            "rep_count": 0,
            "error": f"Video file not found: {video_path}"
        }
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return {
            "rep_count": 0,
            "error": f"Could not open video file: {video_path}. Check file format and codecs."
        }
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {total_frames}")
    
    # If we still have 0x0 dimensions, there's a codec issue
    if frame_width == 0 or frame_height == 0:
        print(f"Error: Could not determine video dimensions. Try converting the video to MP4 format.")
        return {
            "rep_count": 0,
            "error": "Could not determine video dimensions. Try converting to MP4 format."
        }
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or try 'avc1'
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track press state
    rep_count = 0
    press_stage = None  # "up" (arms extended) or "down" (arms at shoulders)
    good_frames = 0
    
    # Lists to store angles and positions for analysis
    elbow_angles_left = []
    elbow_angles_right = []
    shoulder_heights = []  # To track if shoulders stay level
    back_angles = []       # To track back posture (should stay upright)
    wrist_positions = []   # To track wrist alignment with shoulders
    press_heights = []     # To track how high the weights are pressed
    shoulder_rotations = [] # To detect shoulder rotation
    
    frame_count = 0
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get all key points for shoulder press analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Get hip points to measure back angle
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Calculate midpoints for better analysis
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, 
                          (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, 
                      (left_hip[1] + right_hip[1])/2]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Calculate elbow angles (shoulder-elbow-wrist)
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            
            elbow_angles_left.append(left_elbow_angle)
            elbow_angles_right.append(right_elbow_angle)
            
            # Average elbow angle for determining press stage
            avg_elbow_angle = (left_elbow_angle + right_elbow_angle) / 2
            
            # Calculate shoulder height difference to check if shoulders are level
            shoulder_height_diff = abs(left_shoulder[1] - right_shoulder[1])
            shoulder_heights.append(shoulder_height_diff)
            
            # Calculate back angle relative to vertical
            # Create a point directly below mid_hip to represent vertical
            vertical_point = [mid_hip[0], mid_hip[1] + 0.2]  # Adding to y moves down in image coords
            back_angle = calculate_angle(mid_shoulder, mid_hip, vertical_point)
            back_angle = 180 - back_angle if back_angle > 90 else back_angle
            back_angles.append(back_angle)
            
            # Check wrist position relative to shoulders
            # For proper press, wrists should be aligned with or slightly in front of shoulders
            left_wrist_to_shoulder_x = left_wrist[0] - left_shoulder[0]
            right_wrist_to_shoulder_x = right_wrist[0] - right_shoulder[0]
            avg_wrist_position = (left_wrist_to_shoulder_x + right_wrist_to_shoulder_x) / 2
            wrist_positions.append(avg_wrist_position)
            
            # Track press height (wrist height relative to shoulder)
            left_press_height = left_shoulder[1] - left_wrist[1]  # Higher value means higher press
            right_press_height = right_shoulder[1] - right_wrist[1]
            avg_press_height = (left_press_height + right_press_height) / 2
            press_heights.append(avg_press_height)
            
            # Detect shoulder rotation (shoulders should stay level and not rotate forward/backward)
            # Simple proxy: difference in the depth (x-coordinate) of shoulders
            shoulder_rotation = abs(left_shoulder[0] - right_shoulder[0])
            shoulder_rotations.append(shoulder_rotation)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            left_elbow_px = normalized_to_pixel_coordinates(left_elbow[0], left_elbow[1], frame_width, frame_height)
            right_elbow_px = normalized_to_pixel_coordinates(right_elbow[0], right_elbow[1], frame_width, frame_height)
            mid_hip_px = normalized_to_pixel_coordinates(mid_hip[0], mid_hip[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"L: {left_elbow_angle:.1f}°",
                        (left_elbow_px[0] - 50, left_elbow_px[1] + 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"R: {right_elbow_angle:.1f}°",
                        (right_elbow_px[0] - 50, right_elbow_px[1] + 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Back: {back_angle:.1f}°",
                        (mid_hip_px[0] - 50, mid_hip_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine press stage based on elbow angle
            # For shoulder press, "down" is when arms are bent (close to shoulders)
            # "up" is when arms are extended overhead
            
            # Press is "down" when elbow angle is small (arms bent, weights at shoulders)
            if avg_elbow_angle < 90 and (press_stage == "up" or press_stage is None):
                press_stage = "down"
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # Press is "up" when elbow angle is large (arms extended overhead)
            elif avg_elbow_angle > 160 and press_stage == "down":
                press_stage = "up"
                # Coming from "down", count a rep
                rep_count += 1
                print(f"Rep #{rep_count} detected at frame with average elbow angle {avg_elbow_angle:.1f}°")
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path and 'out' in locals():
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    avg_min_elbow_angle = (np.min(elbow_angles_left) + np.min(elbow_angles_right)) / 2 if elbow_angles_left and elbow_angles_right else 180
    avg_max_elbow_angle = (np.max(elbow_angles_left) + np.max(elbow_angles_right)) / 2 if elbow_angles_left and elbow_angles_right else 0
    avg_back_angle = np.mean(back_angles) if back_angles else 0
    
    # Calculate shoulder stability (lower is better)
    shoulder_stability = np.std(shoulder_heights) * 100 if shoulder_heights else 0
    shoulder_stability_score = max(0, 100 - shoulder_stability)  # Convert to 0-100 score
    
    # Calculate wrist position relative to shoulders
    # For proper press, wrists should be aligned with or slightly in front of shoulders
    avg_wrist_position_score = 100 - (abs(np.mean(wrist_positions)) * 300) if wrist_positions else 50
    avg_wrist_position_score = max(0, min(100, avg_wrist_position_score))  # Clamp between 0-100
    
    # Calculate press height consistency
    max_press_height = np.max(press_heights) if press_heights else 0
    
    # Calculate shoulder rotation stability
    rotation_stability = np.std(shoulder_rotations) * 100 if shoulder_rotations else 0
    rotation_stability_score = max(0, 100 - rotation_stability)  # Convert to 0-100 score
    
    # Check for key shoulder press form criteria
    full_range_of_motion = avg_min_elbow_angle < 80 and avg_max_elbow_angle > 160
    back_upright = avg_back_angle < 15  # Back should be very upright in seated press
    shoulders_stable = shoulder_stability_score > 80
    wrists_aligned = avg_wrist_position_score > 70
    shoulders_level = rotation_stability_score > 80
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "min_elbow_angle": avg_min_elbow_angle,  # Lower value means better range of motion at bottom
            "max_elbow_angle": avg_max_elbow_angle,  # Higher value means better extension at top
            "back_angle": avg_back_angle,            # Should be close to 0° (upright)
            "shoulder_stability": shoulder_stability_score,  # Higher is better (0-100)
            "wrist_alignment": avg_wrist_position_score,    # Higher is better (0-100)
            "shoulder_level": rotation_stability_score,     # Higher is better (0-100)
            "max_press_height": max_press_height,           # Higher is better
            "full_range_of_motion": full_range_of_motion,
            "back_upright": back_upright,
            "shoulders_stable": shoulders_stable,
            "wrists_aligned": wrists_aligned,
            "shoulders_level": shoulders_level,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    # Check range of motion
    if avg_min_elbow_angle > 80:
        feedback["feedback"].append("Try to lower the weights more at the bottom of the movement. Your elbows aren't bending enough.")
    
    if avg_max_elbow_angle < 160:
        feedback["feedback"].append("Extend your arms more fully at the top of the press. You're not reaching full extension.")
    
    # Check back position
    if avg_back_angle > 15:
        feedback["feedback"].append("Keep your back more upright against the seat back. You're leaning forward during the press.")
    
    # Check shoulder stability
    if not shoulders_stable:
        feedback["feedback"].append("Focus on keeping your shoulders more stable throughout the movement. Your shoulders are moving excessively.")
    
    # Check wrist alignment
    if not wrists_aligned:
        feedback["feedback"].append("Keep your wrists aligned with your shoulders. Your wrists are drifting too far forward or backward.")
    
    # Check shoulder rotation
    if not shoulders_level:
        feedback["feedback"].append("Keep your shoulders level and avoid rotating them during the press. One shoulder is moving more than the other.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your shoulder press shows good range of motion, proper back position, and stable shoulders.")
        
    return feedback

# Example usage - Make sure you have the correct file path
video_file = "clips/ShoulderPress1.MOV"  # Update this to the correct path
result = analyze_shoulder_press(video_file, "clips/analyzed_press1.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Min elbow angle (lower is better): {result['form_analysis']['min_elbow_angle']:.1f}°")
    print(f"Max elbow angle (higher is better): {result['form_analysis']['max_elbow_angle']:.1f}°")
    print(f"Back angle (should be close to 0°): {result['form_analysis']['back_angle']:.1f}°")
    print(f"Shoulder stability score: {result['form_analysis']['shoulder_stability']:.1f}/100")
    print(f"Wrist alignment score: {result['form_analysis']['wrist_alignment']:.1f}/100")
    print(f"Shoulders level score: {result['form_analysis']['shoulder_level']:.1f}/100")
    print(f"Full range of motion: {'Yes' if result['form_analysis']['full_range_of_motion'] else 'No'}")
    print(f"Back stayed upright: {'Yes' if result['form_analysis']['back_upright'] else 'No'}")
    print(f"Shoulders stayed stable: {'Yes' if result['form_analysis']['shoulders_stable'] else 'No'}")
    print(f"Wrists properly aligned: {'Yes' if result['form_analysis']['wrists_aligned'] else 'No'}")
    print(f"Shoulders stayed level: {'Yes' if result['form_analysis']['shoulders_level'] else 'No'}")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745948235.793811 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745948235.878715 6349325 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745948235.890754 6349325 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29, Total frames: 1102
Detected 33 landmarks
Rep #1 detected at frame with average elbow angle 161.8°
Processing frame 100/1102
Processing frame 200/1102
Rep #2 detected at frame with average elbow angle 163.6°
Processing frame 300/1102
Rep #3 detected at frame with average elbow angle 171.7°
Rep #4 detected at frame with average elbow angle 172.0°
Processing frame 400/1102
Rep #5 detected at frame with average elbow angle 165.7°
Processing frame 500/1102
Processing frame 600/1102
Processing frame 700/1102
Rep #6 detected at frame with average elbow angle 164.2°
Processing frame 800/1102
Rep #7 detected at frame with average elbow angle 165.0°
Processing frame 900/1102
Rep #8 detected at frame with average elbow angle 172.4°
Processing frame 1000/1102
Rep #9 detected at frame with average elbow angle 176.7°
Processing frame 1100/1102

=== ANALYSIS RESULTS ===
Counted 9 reps
Min elbow angle (lower is better): 5.0°
Max elbow angle (higher is better): 179

In [13]:
import mediapipe as mp
import cv2
import numpy as np
import math
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_shoulder_press(video_path, output_video_path=None):
    # Check if file exists
    if not os.path.isfile(video_path):
        print(f"Error: File '{video_path}' does not exist!")
        return {
            "rep_count": 0,
            "error": f"Video file not found: {video_path}"
        }
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return {
            "rep_count": 0,
            "error": f"Could not open video file: {video_path}. Check file format and codecs."
        }
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {total_frames}")
    
    # If we still have 0x0 dimensions, there's a codec issue
    if frame_width == 0 or frame_height == 0:
        print(f"Error: Could not determine video dimensions. Try converting the video to MP4 format.")
        return {
            "rep_count": 0,
            "error": "Could not determine video dimensions. Try converting to MP4 format."
        }
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or try 'avc1'
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track press state
    rep_count = 0
    press_stage = None  # "up" (arms extended) or "down" (arms at shoulders)
    good_frames = 0
    
    # Lists to store angles and positions for analysis
    elbow_angles_left = []
    elbow_angles_right = []
    shoulder_heights = []  # To track if shoulders stay level
    back_angles = []       # To track back posture (should stay upright)
    wrist_positions = []   # To track wrist alignment with shoulders
    press_heights = []     # To track how high the weights are pressed
    shoulder_rotations = [] # To detect shoulder rotation
    
    frame_count = 0
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get all key points for shoulder press analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Get hip points to measure back angle
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Calculate midpoints for better analysis
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, 
                          (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, 
                      (left_hip[1] + right_hip[1])/2]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Calculate elbow angles (shoulder-elbow-wrist)
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            
            elbow_angles_left.append(left_elbow_angle)
            elbow_angles_right.append(right_elbow_angle)
            
            # Average elbow angle for determining press stage
            avg_elbow_angle = (left_elbow_angle + right_elbow_angle) / 2
            
            # Calculate shoulder height difference to check if shoulders are level
            shoulder_height_diff = abs(left_shoulder[1] - right_shoulder[1])
            shoulder_heights.append(shoulder_height_diff)
            
            # Calculate back angle relative to vertical
            # Create a point directly below mid_hip to represent vertical
            vertical_point = [mid_hip[0], mid_hip[1] + 0.2]  # Adding to y moves down in image coords
            back_angle = calculate_angle(mid_shoulder, mid_hip, vertical_point)
            back_angle = 180 - back_angle if back_angle > 90 else back_angle
            back_angles.append(back_angle)
            
            # Check wrist position relative to shoulders
            # For proper press, wrists should be aligned with or slightly in front of shoulders
            left_wrist_to_shoulder_x = left_wrist[0] - left_shoulder[0]
            right_wrist_to_shoulder_x = right_wrist[0] - right_shoulder[0]
            avg_wrist_position = (left_wrist_to_shoulder_x + right_wrist_to_shoulder_x) / 2
            wrist_positions.append(avg_wrist_position)
            
            # Track press height (wrist height relative to shoulder)
            left_press_height = left_shoulder[1] - left_wrist[1]  # Higher value means higher press
            right_press_height = right_shoulder[1] - right_wrist[1]
            avg_press_height = (left_press_height + right_press_height) / 2
            press_heights.append(avg_press_height)
            
            # Detect shoulder rotation (shoulders should stay level and not rotate forward/backward)
            # Simple proxy: difference in the depth (x-coordinate) of shoulders
            shoulder_rotation = abs(left_shoulder[0] - right_shoulder[0])
            shoulder_rotations.append(shoulder_rotation)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            left_elbow_px = normalized_to_pixel_coordinates(left_elbow[0], left_elbow[1], frame_width, frame_height)
            right_elbow_px = normalized_to_pixel_coordinates(right_elbow[0], right_elbow[1], frame_width, frame_height)
            mid_hip_px = normalized_to_pixel_coordinates(mid_hip[0], mid_hip[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"L: {left_elbow_angle:.1f}°",
                        (left_elbow_px[0] - 50, left_elbow_px[1] + 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"R: {right_elbow_angle:.1f}°",
                        (right_elbow_px[0] - 50, right_elbow_px[1] + 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Back: {back_angle:.1f}°",
                        (mid_hip_px[0] - 50, mid_hip_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine press stage based on elbow angle
            # For shoulder press, "down" is when arms are bent (close to shoulders)
            # "up" is when arms are extended overhead
            
            # Press is "down" when elbow angle is small (arms bent, weights at shoulders)
            if avg_elbow_angle < 90 and (press_stage == "up" or press_stage is None):
                press_stage = "down"
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # Press is "up" when elbow angle is large (arms extended overhead)
            elif avg_elbow_angle > 160 and press_stage == "down":
                press_stage = "up"
                # Coming from "down", count a rep
                rep_count += 1
                print(f"Rep #{rep_count} detected at frame with average elbow angle {avg_elbow_angle:.1f}°")
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path and 'out' in locals():
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    avg_min_elbow_angle = (np.min(elbow_angles_left) + np.min(elbow_angles_right)) / 2 if elbow_angles_left and elbow_angles_right else 180
    avg_max_elbow_angle = (np.max(elbow_angles_left) + np.max(elbow_angles_right)) / 2 if elbow_angles_left and elbow_angles_right else 0
    avg_back_angle = np.mean(back_angles) if back_angles else 0
    
    # Calculate shoulder stability (lower is better)
    shoulder_stability = np.std(shoulder_heights) * 100 if shoulder_heights else 0
    shoulder_stability_score = max(0, 100 - shoulder_stability)  # Convert to 0-100 score
    
    # Calculate wrist position relative to shoulders
    # For proper press, wrists should be aligned with or slightly in front of shoulders
    avg_wrist_position_score = 100 - (abs(np.mean(wrist_positions)) * 300) if wrist_positions else 50
    avg_wrist_position_score = max(0, min(100, avg_wrist_position_score))  # Clamp between 0-100
    
    # Calculate press height consistency
    max_press_height = np.max(press_heights) if press_heights else 0
    
    # Calculate shoulder rotation stability
    rotation_stability = np.std(shoulder_rotations) * 100 if shoulder_rotations else 0
    rotation_stability_score = max(0, 100 - rotation_stability)  # Convert to 0-100 score
    
    # Check for key shoulder press form criteria
    full_range_of_motion = avg_min_elbow_angle < 80 and avg_max_elbow_angle > 160
    back_upright = avg_back_angle < 15  # Back should be very upright in seated press
    shoulders_stable = shoulder_stability_score > 80
    wrists_aligned = avg_wrist_position_score > 70
    shoulders_level = rotation_stability_score > 80
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "min_elbow_angle": avg_min_elbow_angle,  # Lower value means better range of motion at bottom
            "max_elbow_angle": avg_max_elbow_angle,  # Higher value means better extension at top
            "back_angle": avg_back_angle,            # Should be close to 0° (upright)
            "shoulder_stability": shoulder_stability_score,  # Higher is better (0-100)
            "wrist_alignment": avg_wrist_position_score,    # Higher is better (0-100)
            "shoulder_level": rotation_stability_score,     # Higher is better (0-100)
            "max_press_height": max_press_height,           # Higher is better
            "full_range_of_motion": full_range_of_motion,
            "back_upright": back_upright,
            "shoulders_stable": shoulders_stable,
            "wrists_aligned": wrists_aligned,
            "shoulders_level": shoulders_level,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    # Check range of motion
    if avg_min_elbow_angle > 80:
        feedback["feedback"].append("Try to lower the weights more at the bottom of the movement. Your elbows aren't bending enough.")
    
    if avg_max_elbow_angle < 160:
        feedback["feedback"].append("Extend your arms more fully at the top of the press. You're not reaching full extension.")
    
    # Check back position
    if avg_back_angle > 15:
        feedback["feedback"].append("Keep your back more upright against the seat back. You're leaning forward during the press.")
    
    # Check shoulder stability
    if not shoulders_stable:
        feedback["feedback"].append("Focus on keeping your shoulders more stable throughout the movement. Your shoulders are moving excessively.")
    
    # Check wrist alignment
    if not wrists_aligned:
        feedback["feedback"].append("Keep your wrists aligned with your shoulders. Your wrists are drifting too far forward or backward.")
    
    # Check shoulder rotation
    if not shoulders_level:
        feedback["feedback"].append("Keep your shoulders level and avoid rotating them during the press. One shoulder is moving more than the other.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your shoulder press shows good range of motion, proper back position, and stable shoulders.")
        
    return feedback

# Example usage - Make sure you have the correct file path
video_file = "clips/ShoulderPress2.MOV"  # Update this to the correct path
result = analyze_shoulder_press(video_file, "clips/analyzed_press2.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Min elbow angle (lower is better): {result['form_analysis']['min_elbow_angle']:.1f}°")
    print(f"Max elbow angle (higher is better): {result['form_analysis']['max_elbow_angle']:.1f}°")
    print(f"Back angle (should be close to 0°): {result['form_analysis']['back_angle']:.1f}°")
    print(f"Shoulder stability score: {result['form_analysis']['shoulder_stability']:.1f}/100")
    print(f"Wrist alignment score: {result['form_analysis']['wrist_alignment']:.1f}/100")
    print(f"Shoulders level score: {result['form_analysis']['shoulder_level']:.1f}/100")
    print(f"Full range of motion: {'Yes' if result['form_analysis']['full_range_of_motion'] else 'No'}")
    print(f"Back stayed upright: {'Yes' if result['form_analysis']['back_upright'] else 'No'}")
    print(f"Shoulders stayed stable: {'Yes' if result['form_analysis']['shoulders_stable'] else 'No'}")
    print(f"Wrists properly aligned: {'Yes' if result['form_analysis']['wrists_aligned'] else 'No'}")
    print(f"Shoulders stayed level: {'Yes' if result['form_analysis']['shoulders_level'] else 'No'}")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745948291.839619 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745948291.933619 6350895 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745948291.945874 6350901 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29, Total frames: 435
Detected 33 landmarks
Rep #1 detected at frame with average elbow angle 166.6°
Processing frame 100/435
Rep #2 detected at frame with average elbow angle 162.3°
Processing frame 200/435
Processing frame 300/435
Processing frame 400/435
Rep #3 detected at frame with average elbow angle 162.0°

=== ANALYSIS RESULTS ===
Counted 3 reps
Min elbow angle (lower is better): 4.5°
Max elbow angle (higher is better): 172.7°
Back angle (should be close to 0°): 40.8°
Shoulder stability score: 96.9/100
Wrist alignment score: 56.4/100
Shoulders level score: 98.4/100
Full range of motion: Yes
Back stayed upright: No
Shoulders stayed stable: Yes
Wrists properly aligned: No
Shoulders stayed level: Yes
Frames analyzed: 435

Feedback:
- Keep your back more upright against the seat back. You're leaning forward during the press.
- Keep your wrists aligned with your shoulders. Your wrists are drifting too far forward or backward.


In [14]:
import mediapipe as mp
import cv2
import numpy as np
import math
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_step_up(video_path, output_video_path=None):
    # Check if file exists
    if not os.path.isfile(video_path):
        print(f"Error: File '{video_path}' does not exist!")
        return {
            "rep_count": 0,
            "error": f"Video file not found: {video_path}"
        }
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return {
            "rep_count": 0,
            "error": f"Could not open video file: {video_path}. Check file format and codecs."
        }
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {total_frames}")
    
    # If we still have 0x0 dimensions, there's a codec issue
    if frame_width == 0 or frame_height == 0:
        print(f"Error: Could not determine video dimensions. Try converting the video to MP4 format.")
        return {
            "rep_count": 0,
            "error": "Could not determine video dimensions. Try converting to MP4 format."
        }
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or try 'avc1'
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track step-up state
    rep_count = 0
    step_stage = None  # "up" (standing on box) or "down" (on ground)
    good_frames = 0
    
    # Determine which leg is the lead leg by tracking movements
    active_leg = None  # "left" or "right"
    left_knee_positions = []
    right_knee_positions = []
    
    # Lists to store angles and positions for analysis
    lead_knee_angles = []    # Lead knee should be around 90° at bottom of movement
    trail_knee_angles = []   # Trail leg knee angles
    hip_heights = []         # Hip height relative to step to track full extension
    torso_angles = []        # Torso should stay upright
    knee_alignments = []     # Lead knee should track over ankle
    hip_stability = []       # Hip drop/rotation
    lead_ankle_angles = []   # Ankle mobility on lead leg
    
    frame_count = 0
    start_height = None      # To store initial height for comparison
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get all key points for step-up analysis
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
            
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            
            # Calculate midpoints
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, 
                          (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, 
                      (left_hip[1] + right_hip[1])/2]
            
            # Track knee positions to determine lead leg
            left_knee_positions.append(left_knee)
            right_knee_positions.append(right_knee)
            
            # If we have enough frames, determine which leg is the lead leg
            if good_frames == 30 and active_leg is None:
                # Calculate vertical movement (y-coordinate) for each knee
                left_movement = 0
                right_movement = 0
                
                for i in range(1, len(left_knee_positions)):
                    # We focus on y movement (vertical) for step-ups
                    left_movement += abs(left_knee_positions[i][1] - left_knee_positions[i-1][1])
                    right_movement += abs(right_knee_positions[i][1] - right_knee_positions[i-1][1])
                
                # The leg with more vertical movement is likely the lead leg
                if left_movement > right_movement:
                    active_leg = "left"
                else:
                    active_leg = "right"
                    
                print(f"Detected {active_leg} leg as the lead leg for step-ups")
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Determine lead and trail legs based on detected active leg
            if active_leg == "left" or active_leg is None:
                lead_hip = left_hip
                lead_knee = left_knee
                lead_ankle = left_ankle
                lead_foot_index = left_foot_index
                
                trail_hip = right_hip
                trail_knee = right_knee
                trail_ankle = right_ankle
                trail_foot_index = right_foot_index
                
                lead_label = "L"
                trail_label = "R"
            else:
                lead_hip = right_hip
                lead_knee = right_knee
                lead_ankle = right_ankle
                lead_foot_index = right_foot_index
                
                trail_hip = left_hip
                trail_knee = left_knee
                trail_ankle = left_ankle
                trail_foot_index = left_foot_index
                
                lead_label = "R"
                trail_label = "L"
            
            # Calculate knee angles (hip-knee-ankle)
            lead_knee_angle = calculate_angle(lead_hip, lead_knee, lead_ankle)
            trail_knee_angle = calculate_angle(trail_hip, trail_knee, trail_ankle)
            
            lead_knee_angles.append(lead_knee_angle)
            trail_knee_angles.append(trail_knee_angle)
            
            # Calculate ankle angle (knee-ankle-foot) for lead leg
            lead_ankle_angle = calculate_angle(lead_knee, lead_ankle, lead_foot_index)
            lead_ankle_angles.append(lead_ankle_angle)
            
            # Calculate torso angle relative to vertical
            # Create a point directly below mid_hip to represent vertical
            vertical_point = [mid_hip[0], mid_hip[1] + 0.2]  # Adding to y moves down in image coords
            torso_angle = calculate_angle(mid_shoulder, mid_hip, vertical_point)
            torso_angle = 180 - torso_angle if torso_angle > 90 else torso_angle
            torso_angles.append(torso_angle)
            
            # Track hip height for full extension
            hip_height = mid_hip[1]  # Lower y-value means higher position in image coordinates
            hip_heights.append(hip_height)
            
            # Store starting height for comparison if it's not set yet
            if start_height is None and good_frames > 5:  # Wait a few frames for stability
                start_height = hip_height
            
            # Calculate knee alignment (should track over ankle)
            lead_knee_to_ankle_x = lead_knee[0] - lead_ankle[0]
            knee_alignments.append(lead_knee_to_ankle_x)
            
            # Calculate hip stability (measure level of hips)
            hip_level_diff = abs(left_hip[1] - right_hip[1])
            hip_stability.append(hip_level_diff)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            lead_knee_px = normalized_to_pixel_coordinates(lead_knee[0], lead_knee[1], frame_width, frame_height)
            trail_knee_px = normalized_to_pixel_coordinates(trail_knee[0], trail_knee[1], frame_width, frame_height)
            mid_hip_px = normalized_to_pixel_coordinates(mid_hip[0], mid_hip[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"{lead_label} Knee: {lead_knee_angle:.1f}°",
                        (lead_knee_px[0] - 70, lead_knee_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Torso: {torso_angle:.1f}°",
                        (mid_hip_px[0] - 50, mid_hip_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine step stage based on hip height relative to starting position
            # For step-ups, we need to track position more than angles
            
            # Use lowest observed hip height for reference (higher value in image coords)
            if start_height is not None:
                # If we're significantly higher than starting position, we're in "up" stage
                if hip_height < (start_height - 0.05) and (step_stage == "down" or step_stage is None):
                    step_stage = "up"
                    cv2.putText(annotated_image, 'UP', (50, 60), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
                # If we're close to or below starting position, we're in "down" stage
                elif hip_height > (start_height - 0.02) and step_stage == "up":
                    step_stage = "down"
                    # Coming from "up", count a rep
                    rep_count += 1
                    print(f"Rep #{rep_count} detected at frame with hip height {hip_height:.3f}")
                    cv2.putText(annotated_image, 'DOWN', (50, 60), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path and 'out' in locals():
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    min_lead_knee_angle = np.min(lead_knee_angles) if lead_knee_angles else 180
    avg_torso_angle = np.mean(torso_angles) if torso_angles else 0
    
    # Calculate max hip height change (to check if full extension at top)
    if hip_heights and start_height is not None:
        min_hip_height = np.min(hip_heights)  # Highest position (lowest y-value in image)
        hip_height_change = start_height - min_hip_height
    else:
        hip_height_change = 0
    
    # Calculate knee alignment score
    knee_alignment_avg = np.mean(abs(np.array(knee_alignments))) if knee_alignments else 0
    knee_alignment_score = 100 - (knee_alignment_avg * 300)  # Convert to 0-100 score
    knee_alignment_score = max(0, min(100, knee_alignment_score))  # Clamp between 0-100
    
    # Calculate hip stability score
    hip_stability_avg = np.mean(hip_stability) if hip_stability else 0
    hip_stability_score = 100 - (hip_stability_avg * 500)  # Convert to 0-100 score
    hip_stability_score = max(0, min(100, hip_stability_score))  # Clamp between 0-100
    
    # Calculate ankle mobility score
    ankle_mobility = np.min(lead_ankle_angles) if lead_ankle_angles else 90
    # Lower ankle angle indicates better dorsiflexion
    ankle_mobility_score = 100 - ((ankle_mobility - 70) * 3)  # Convert to 0-100 score
    ankle_mobility_score = max(0, min(100, ankle_mobility_score))  # Clamp between 0-100
    
    # Check for key step-up form criteria
    proper_knee_angle = 70 <= min_lead_knee_angle <= 110  # Should bend to ~90° at bottom
    full_extension = hip_height_change > 0.08  # Sufficient height change indicates full extension
    torso_upright = avg_torso_angle < 20  # Torso should stay relatively vertical
    proper_knee_alignment = knee_alignment_score > 70  # Knee should track over ankle
    good_hip_stability = hip_stability_score > 70  # Hips should stay level
    good_ankle_mobility = ankle_mobility_score > 60  # Sufficient ankle mobility
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "lead_knee_angle": min_lead_knee_angle,  # Should be ~90° at bottom
            "torso_angle": avg_torso_angle,          # Should stay upright (<20°)
            "hip_extension": hip_height_change,      # Higher is better (full extension)
            "knee_alignment": knee_alignment_score,  # Higher is better (0-100)
            "hip_stability": hip_stability_score,    # Higher is better (0-100)
            "ankle_mobility": ankle_mobility_score,  # Higher is better (0-100)
            "proper_knee_angle": proper_knee_angle,
            "full_extension": full_extension,
            "torso_upright": torso_upright,
            "proper_knee_alignment": proper_knee_alignment,
            "good_hip_stability": good_hip_stability,
            "good_ankle_mobility": good_ankle_mobility,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    # Check knee angle
    if min_lead_knee_angle > 110:
        feedback["feedback"].append("Your knee isn't bending enough as you step up. Try a higher step or focus on a deeper knee bend.")
    elif min_lead_knee_angle < 70:
        feedback["feedback"].append("Your knee is bending too much. This might indicate the step is too high or you're leaning too far forward.")
    
    # Check torso position
    if avg_torso_angle > 20:
        feedback["feedback"].append("Try to keep your torso more upright. You're leaning forward too much during the step-up.")
    
    # Check hip extension
    if not full_extension:
        feedback["feedback"].append("Extend your hips fully at the top of the movement. Stand tall on the step before lowering back down.")
    
    # Check knee alignment
    if not proper_knee_alignment:
        feedback["feedback"].append("Focus on keeping your knee aligned over your ankle. Your knee is tracking too far inward or outward.")
    
    # Check hip stability
    if not good_hip_stability:
        feedback["feedback"].append("Keep your hips level throughout the movement. Your hips are dropping or rotating to one side.")
    
    # Check ankle mobility
    if not good_ankle_mobility:
        feedback["feedback"].append("Work on ankle mobility. Limited ankle dorsiflexion is affecting your step-up form.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your step-ups show good knee alignment, proper torso position, and full extension at the top.")
        
    return feedback

# Example usage - Make sure you have the correct file path
video_file = "clips/StepUp1.MOV"  # Update this to the correct path
result = analyze_step_up(video_file, "clips/analyzed_stepup1.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Lead knee angle (ideal ~90°): {result['form_analysis']['lead_knee_angle']:.1f}°")
    print(f"Torso angle (lower is better): {result['form_analysis']['torso_angle']:.1f}°")
    print(f"Hip extension: {result['form_analysis']['hip_extension']:.3f}")
    print(f"Knee alignment score: {result['form_analysis']['knee_alignment']:.1f}/100")
    print(f"Hip stability score: {result['form_analysis']['hip_stability']:.1f}/100")
    print(f"Ankle mobility score: {result['form_analysis']['ankle_mobility']:.1f}/100")
    print(f"Proper knee angle: {'Yes' if result['form_analysis']['proper_knee_angle'] else 'No'}")
    print(f"Full extension at top: {'Yes' if result['form_analysis']['full_extension'] else 'No'}")
    print(f"Torso stayed upright: {'Yes' if result['form_analysis']['torso_upright'] else 'No'}")
    print(f"Proper knee alignment: {'Yes' if result['form_analysis']['proper_knee_alignment'] else 'No'}")
    print(f"Good hip stability: {'Yes' if result['form_analysis']['good_hip_stability'] else 'No'}")
    print(f"Good ankle mobility: {'Yes' if result['form_analysis']['good_ankle_mobility'] else 'No'}")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745948483.088899 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745948483.206086 6356599 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745948483.219770 6356600 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29, Total frames: 725
Detected 33 landmarks
Detected left leg as the lead leg for step-ups
Processing frame 100/725
Rep #1 detected at frame with hip height 0.690
Processing frame 200/725
Rep #2 detected at frame with hip height 0.687
Processing frame 300/725
Rep #3 detected at frame with hip height 0.690
Processing frame 400/725
Processing frame 500/725
Processing frame 600/725
Processing frame 700/725
Rep #4 detected at frame with hip height 0.686

=== ANALYSIS RESULTS ===
Counted 4 reps
Lead knee angle (ideal ~90°): 29.5°
Torso angle (lower is better): 71.5°
Hip extension: 0.247
Knee alignment score: 74.6/100
Hip stability score: 90.5/100
Ankle mobility score: 100.0/100
Proper knee angle: No
Full extension at top: Yes
Torso stayed upright: No
Proper knee alignment: Yes
Good hip stability: Yes
Good ankle mobility: Yes
Frames analyzed: 725

Feedback:
- Your knee is bending too much. This might indicate the step is too high or you're leaning too far fo

In [15]:
import mediapipe as mp
import cv2
import numpy as np
import math
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_step_up(video_path, output_video_path=None):
    # Check if file exists
    if not os.path.isfile(video_path):
        print(f"Error: File '{video_path}' does not exist!")
        return {
            "rep_count": 0,
            "error": f"Video file not found: {video_path}"
        }
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return {
            "rep_count": 0,
            "error": f"Could not open video file: {video_path}. Check file format and codecs."
        }
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {total_frames}")
    
    # If we still have 0x0 dimensions, there's a codec issue
    if frame_width == 0 or frame_height == 0:
        print(f"Error: Could not determine video dimensions. Try converting the video to MP4 format.")
        return {
            "rep_count": 0,
            "error": "Could not determine video dimensions. Try converting to MP4 format."
        }
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or try 'avc1'
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track step-up state
    rep_count = 0
    step_stage = None  # "up" (standing on box) or "down" (on ground)
    good_frames = 0
    
    # Determine which leg is the lead leg by tracking movements
    active_leg = None  # "left" or "right"
    left_knee_positions = []
    right_knee_positions = []
    
    # Lists to store angles and positions for analysis
    lead_knee_angles = []    # Lead knee should be around 90° at bottom of movement
    trail_knee_angles = []   # Trail leg knee angles
    hip_heights = []         # Hip height relative to step to track full extension
    torso_angles = []        # Torso should stay upright
    knee_alignments = []     # Lead knee should track over ankle
    hip_stability = []       # Hip drop/rotation
    lead_ankle_angles = []   # Ankle mobility on lead leg
    
    frame_count = 0
    start_height = None      # To store initial height for comparison
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get all key points for step-up analysis
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
            
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            
            # Calculate midpoints
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, 
                          (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, 
                      (left_hip[1] + right_hip[1])/2]
            
            # Track knee positions to determine lead leg
            left_knee_positions.append(left_knee)
            right_knee_positions.append(right_knee)
            
            # If we have enough frames, determine which leg is the lead leg
            if good_frames == 30 and active_leg is None:
                # Calculate vertical movement (y-coordinate) for each knee
                left_movement = 0
                right_movement = 0
                
                for i in range(1, len(left_knee_positions)):
                    # We focus on y movement (vertical) for step-ups
                    left_movement += abs(left_knee_positions[i][1] - left_knee_positions[i-1][1])
                    right_movement += abs(right_knee_positions[i][1] - right_knee_positions[i-1][1])
                
                # The leg with more vertical movement is likely the lead leg
                if left_movement > right_movement:
                    active_leg = "left"
                else:
                    active_leg = "right"
                    
                print(f"Detected {active_leg} leg as the lead leg for step-ups")
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Determine lead and trail legs based on detected active leg
            if active_leg == "left" or active_leg is None:
                lead_hip = left_hip
                lead_knee = left_knee
                lead_ankle = left_ankle
                lead_foot_index = left_foot_index
                
                trail_hip = right_hip
                trail_knee = right_knee
                trail_ankle = right_ankle
                trail_foot_index = right_foot_index
                
                lead_label = "L"
                trail_label = "R"
            else:
                lead_hip = right_hip
                lead_knee = right_knee
                lead_ankle = right_ankle
                lead_foot_index = right_foot_index
                
                trail_hip = left_hip
                trail_knee = left_knee
                trail_ankle = left_ankle
                trail_foot_index = left_foot_index
                
                lead_label = "R"
                trail_label = "L"
            
            # Calculate knee angles (hip-knee-ankle)
            lead_knee_angle = calculate_angle(lead_hip, lead_knee, lead_ankle)
            trail_knee_angle = calculate_angle(trail_hip, trail_knee, trail_ankle)
            
            lead_knee_angles.append(lead_knee_angle)
            trail_knee_angles.append(trail_knee_angle)
            
            # Calculate ankle angle (knee-ankle-foot) for lead leg
            lead_ankle_angle = calculate_angle(lead_knee, lead_ankle, lead_foot_index)
            lead_ankle_angles.append(lead_ankle_angle)
            
            # Calculate torso angle relative to vertical
            # Create a point directly below mid_hip to represent vertical
            vertical_point = [mid_hip[0], mid_hip[1] + 0.2]  # Adding to y moves down in image coords
            torso_angle = calculate_angle(mid_shoulder, mid_hip, vertical_point)
            torso_angle = 180 - torso_angle if torso_angle > 90 else torso_angle
            torso_angles.append(torso_angle)
            
            # Track hip height for full extension
            hip_height = mid_hip[1]  # Lower y-value means higher position in image coordinates
            hip_heights.append(hip_height)
            
            # Store starting height for comparison if it's not set yet
            if start_height is None and good_frames > 5:  # Wait a few frames for stability
                start_height = hip_height
            
            # Calculate knee alignment (should track over ankle)
            lead_knee_to_ankle_x = lead_knee[0] - lead_ankle[0]
            knee_alignments.append(lead_knee_to_ankle_x)
            
            # Calculate hip stability (measure level of hips)
            hip_level_diff = abs(left_hip[1] - right_hip[1])
            hip_stability.append(hip_level_diff)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            lead_knee_px = normalized_to_pixel_coordinates(lead_knee[0], lead_knee[1], frame_width, frame_height)
            trail_knee_px = normalized_to_pixel_coordinates(trail_knee[0], trail_knee[1], frame_width, frame_height)
            mid_hip_px = normalized_to_pixel_coordinates(mid_hip[0], mid_hip[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"{lead_label} Knee: {lead_knee_angle:.1f}°",
                        (lead_knee_px[0] - 70, lead_knee_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Torso: {torso_angle:.1f}°",
                        (mid_hip_px[0] - 50, mid_hip_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine step stage based on hip height relative to starting position
            # For step-ups, we need to track position more than angles
            
            # Use lowest observed hip height for reference (higher value in image coords)
            if start_height is not None:
                # If we're significantly higher than starting position, we're in "up" stage
                if hip_height < (start_height - 0.05) and (step_stage == "down" or step_stage is None):
                    step_stage = "up"
                    cv2.putText(annotated_image, 'UP', (50, 60), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
                # If we're close to or below starting position, we're in "down" stage
                elif hip_height > (start_height - 0.02) and step_stage == "up":
                    step_stage = "down"
                    # Coming from "up", count a rep
                    rep_count += 1
                    print(f"Rep #{rep_count} detected at frame with hip height {hip_height:.3f}")
                    cv2.putText(annotated_image, 'DOWN', (50, 60), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path and 'out' in locals():
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    min_lead_knee_angle = np.min(lead_knee_angles) if lead_knee_angles else 180
    avg_torso_angle = np.mean(torso_angles) if torso_angles else 0
    
    # Calculate max hip height change (to check if full extension at top)
    if hip_heights and start_height is not None:
        min_hip_height = np.min(hip_heights)  # Highest position (lowest y-value in image)
        hip_height_change = start_height - min_hip_height
    else:
        hip_height_change = 0
    
    # Calculate knee alignment score
    knee_alignment_avg = np.mean(abs(np.array(knee_alignments))) if knee_alignments else 0
    knee_alignment_score = 100 - (knee_alignment_avg * 300)  # Convert to 0-100 score
    knee_alignment_score = max(0, min(100, knee_alignment_score))  # Clamp between 0-100
    
    # Calculate hip stability score
    hip_stability_avg = np.mean(hip_stability) if hip_stability else 0
    hip_stability_score = 100 - (hip_stability_avg * 500)  # Convert to 0-100 score
    hip_stability_score = max(0, min(100, hip_stability_score))  # Clamp between 0-100
    
    # Calculate ankle mobility score
    ankle_mobility = np.min(lead_ankle_angles) if lead_ankle_angles else 90
    # Lower ankle angle indicates better dorsiflexion
    ankle_mobility_score = 100 - ((ankle_mobility - 70) * 3)  # Convert to 0-100 score
    ankle_mobility_score = max(0, min(100, ankle_mobility_score))  # Clamp between 0-100
    
    # Check for key step-up form criteria
    proper_knee_angle = 70 <= min_lead_knee_angle <= 110  # Should bend to ~90° at bottom
    full_extension = hip_height_change > 0.08  # Sufficient height change indicates full extension
    torso_upright = avg_torso_angle < 20  # Torso should stay relatively vertical
    proper_knee_alignment = knee_alignment_score > 70  # Knee should track over ankle
    good_hip_stability = hip_stability_score > 70  # Hips should stay level
    good_ankle_mobility = ankle_mobility_score > 60  # Sufficient ankle mobility
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "lead_knee_angle": min_lead_knee_angle,  # Should be ~90° at bottom
            "torso_angle": avg_torso_angle,          # Should stay upright (<20°)
            "hip_extension": hip_height_change,      # Higher is better (full extension)
            "knee_alignment": knee_alignment_score,  # Higher is better (0-100)
            "hip_stability": hip_stability_score,    # Higher is better (0-100)
            "ankle_mobility": ankle_mobility_score,  # Higher is better (0-100)
            "proper_knee_angle": proper_knee_angle,
            "full_extension": full_extension,
            "torso_upright": torso_upright,
            "proper_knee_alignment": proper_knee_alignment,
            "good_hip_stability": good_hip_stability,
            "good_ankle_mobility": good_ankle_mobility,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    # Check knee angle
    if min_lead_knee_angle > 110:
        feedback["feedback"].append("Your knee isn't bending enough as you step up. Try a higher step or focus on a deeper knee bend.")
    elif min_lead_knee_angle < 70:
        feedback["feedback"].append("Your knee is bending too much. This might indicate the step is too high or you're leaning too far forward.")
    
    # Check torso position
    if avg_torso_angle > 20:
        feedback["feedback"].append("Try to keep your torso more upright. You're leaning forward too much during the step-up.")
    
    # Check hip extension
    if not full_extension:
        feedback["feedback"].append("Extend your hips fully at the top of the movement. Stand tall on the step before lowering back down.")
    
    # Check knee alignment
    if not proper_knee_alignment:
        feedback["feedback"].append("Focus on keeping your knee aligned over your ankle. Your knee is tracking too far inward or outward.")
    
    # Check hip stability
    if not good_hip_stability:
        feedback["feedback"].append("Keep your hips level throughout the movement. Your hips are dropping or rotating to one side.")
    
    # Check ankle mobility
    if not good_ankle_mobility:
        feedback["feedback"].append("Work on ankle mobility. Limited ankle dorsiflexion is affecting your step-up form.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your step-ups show good knee alignment, proper torso position, and full extension at the top.")
        
    return feedback

# Example usage - Make sure you have the correct file path
video_file = "clips/StepUp2.MOV"  # Update this to the correct path
result = analyze_step_up(video_file, "clips/analyzed_stepup2.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Lead knee angle (ideal ~90°): {result['form_analysis']['lead_knee_angle']:.1f}°")
    print(f"Torso angle (lower is better): {result['form_analysis']['torso_angle']:.1f}°")
    print(f"Hip extension: {result['form_analysis']['hip_extension']:.3f}")
    print(f"Knee alignment score: {result['form_analysis']['knee_alignment']:.1f}/100")
    print(f"Hip stability score: {result['form_analysis']['hip_stability']:.1f}/100")
    print(f"Ankle mobility score: {result['form_analysis']['ankle_mobility']:.1f}/100")
    print(f"Proper knee angle: {'Yes' if result['form_analysis']['proper_knee_angle'] else 'No'}")
    print(f"Full extension at top: {'Yes' if result['form_analysis']['full_extension'] else 'No'}")
    print(f"Torso stayed upright: {'Yes' if result['form_analysis']['torso_upright'] else 'No'}")
    print(f"Proper knee alignment: {'Yes' if result['form_analysis']['proper_knee_alignment'] else 'No'}")
    print(f"Good hip stability: {'Yes' if result['form_analysis']['good_hip_stability'] else 'No'}")
    print(f"Good ankle mobility: {'Yes' if result['form_analysis']['good_ankle_mobility'] else 'No'}")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745948518.565481 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745948518.642722 6357974 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745948518.654726 6357984 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29, Total frames: 525
Detected 33 landmarks
Detected left leg as the lead leg for step-ups
Processing frame 100/525
Rep #1 detected at frame with hip height 0.763
Processing frame 200/525
Rep #2 detected at frame with hip height 0.754
Processing frame 300/525
Rep #3 detected at frame with hip height 0.751
Processing frame 400/525
Processing frame 500/525
Rep #4 detected at frame with hip height 0.751

=== ANALYSIS RESULTS ===
Counted 4 reps
Lead knee angle (ideal ~90°): 21.8°
Torso angle (lower is better): 77.7°
Hip extension: 0.407
Knee alignment score: 58.0/100
Hip stability score: 80.9/100
Ankle mobility score: 100.0/100
Proper knee angle: No
Full extension at top: Yes
Torso stayed upright: No
Proper knee alignment: No
Good hip stability: Yes
Good ankle mobility: Yes
Frames analyzed: 525

Feedback:
- Your knee is bending too much. This might indicate the step is too high or you're leaning too far forward.
- Try to keep your torso more upright. You'r

In [16]:
import mediapipe as mp
import cv2
import numpy as np
import math
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_tricep_extension(video_path, output_video_path=None):
    # Check if file exists
    if not os.path.isfile(video_path):
        print(f"Error: File '{video_path}' does not exist!")
        return {
            "rep_count": 0,
            "error": f"Video file not found: {video_path}"
        }
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return {
            "rep_count": 0,
            "error": f"Could not open video file: {video_path}. Check file format and codecs."
        }
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {total_frames}")
    
    # If we still have 0x0 dimensions, there's a codec issue
    if frame_width == 0 or frame_height == 0:
        print(f"Error: Could not determine video dimensions. Try converting the video to MP4 format.")
        return {
            "rep_count": 0,
            "error": "Could not determine video dimensions. Try converting to MP4 format."
        }
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or try 'avc1'
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track tricep extension state
    rep_count = 0
    extension_stage = None  # "up" (arms extended) or "down" (arms bent)
    good_frames = 0
    
    # Determine if it's two-arm or one-arm extension by analyzing initial frames
    exercise_type = None    # "single" or "double"
    active_arm = None       # "left" or "right" (for single-arm only)
    
    # Lists to store angles and positions for analysis
    elbow_angles_left = []
    elbow_angles_right = []
    shoulder_angles_left = [] # To check if upper arm stays vertical/stationary
    shoulder_angles_right = []
    wrist_alignments = []   # To track wrist alignment with forearms
    torso_angles = []       # To track torso stability
    head_positions = []     # To track head position
    
    frame_count = 0
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get all key points for tricep extension analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Get hip and head points for posture analysis
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                   landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            
            # Calculate midpoints
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, 
                          (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, 
                      (left_hip[1] + right_hip[1])/2]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Calculate elbow angles (shoulder-elbow-wrist)
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            
            elbow_angles_left.append(left_elbow_angle)
            elbow_angles_right.append(right_elbow_angle)
            
            # Calculate shoulder angles to check if upper arm stays vertical
            # Create vertical reference points directly below shoulders
            left_vertical_ref = [left_shoulder[0], left_shoulder[1] + 0.2]
            right_vertical_ref = [right_shoulder[0], right_shoulder[1] + 0.2]
            
            # Calculate angle between shoulder, elbow, and vertical reference
            left_shoulder_angle = calculate_angle(left_vertical_ref, left_shoulder, left_elbow)
            right_shoulder_angle = calculate_angle(right_vertical_ref, right_shoulder, right_elbow)
            
            shoulder_angles_left.append(left_shoulder_angle)
            shoulder_angles_right.append(right_shoulder_angle)
            
            # Calculate torso angle relative to vertical
            # Create a point directly below mid_hip to represent vertical
            vertical_point = [mid_hip[0], mid_hip[1] + 0.2]  # Adding to y moves down in image coords
            torso_angle = calculate_angle(mid_shoulder, mid_hip, vertical_point)
            torso_angle = 180 - torso_angle if torso_angle > 90 else torso_angle
            torso_angles.append(torso_angle)
            
            # Track head position relative to shoulders
            head_to_shoulder_distance = math.sqrt((nose[0] - mid_shoulder[0])**2 + 
                                                 (nose[1] - mid_shoulder[1])**2)
            head_positions.append(head_to_shoulder_distance)
            
            # Determine exercise type based on elbow movement in first 30 frames
            if good_frames == 30 and exercise_type is None:
                left_range = max(elbow_angles_left) - min(elbow_angles_left)
                right_range = max(elbow_angles_right) - min(elbow_angles_right)
                
                # If both elbows have significant movement, it's double-arm
                if left_range > 20 and right_range > 20:
                    exercise_type = "double"
                elif left_range > right_range:
                    exercise_type = "single"
                    active_arm = "left"
                else:
                    exercise_type = "single"
                    active_arm = "right"
                    
                print(f"Detected {exercise_type}-arm tricep extension with {active_arm if exercise_type == 'single' else 'both'} arm(s)")
            
            # Calculate average wrist alignment
            # For tricep extensions, wrists should stay aligned with forearms
            left_wrist_alignment = abs(left_elbow_angle - 180) if left_elbow_angle > 90 else left_elbow_angle
            right_wrist_alignment = abs(right_elbow_angle - 180) if right_elbow_angle > 90 else right_elbow_angle
            
            if exercise_type == "double":
                avg_wrist_alignment = (left_wrist_alignment + right_wrist_alignment) / 2
            elif active_arm == "left":
                avg_wrist_alignment = left_wrist_alignment
            else:
                avg_wrist_alignment = right_wrist_alignment
                
            wrist_alignments.append(avg_wrist_alignment)
            
            # Determine which elbow angle to use for rep counting
            if exercise_type == "double":
                avg_elbow_angle = (left_elbow_angle + right_elbow_angle) / 2
            elif active_arm == "left":
                avg_elbow_angle = left_elbow_angle
            else:
                avg_elbow_angle = right_elbow_angle
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            left_elbow_px = normalized_to_pixel_coordinates(left_elbow[0], left_elbow[1], frame_width, frame_height)
            right_elbow_px = normalized_to_pixel_coordinates(right_elbow[0], right_elbow[1], frame_width, frame_height)
            mid_hip_px = normalized_to_pixel_coordinates(mid_hip[0], mid_hip[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"L: {left_elbow_angle:.1f}°",
                        (left_elbow_px[0] - 50, left_elbow_px[1] + 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"R: {right_elbow_angle:.1f}°",
                        (right_elbow_px[0] - 50, right_elbow_px[1] + 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Torso: {torso_angle:.1f}°",
                        (mid_hip_px[0] - 50, mid_hip_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine extension stage based on elbow angle
            # For tricep extensions, "down" is when elbows are bent
            # "up" is when arms are extended
            
            # Extension is "down" when elbow angle is small (arms bent, weight behind head)
            if avg_elbow_angle < 90 and (extension_stage == "up" or extension_stage is None):
                extension_stage = "down"
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # Extension is "up" when elbow angle is large (arms extended)
            elif avg_elbow_angle > 160 and extension_stage == "down":
                extension_stage = "up"
                # Coming from "down", count a rep
                rep_count += 1
                print(f"Rep #{rep_count} detected at frame with average elbow angle {avg_elbow_angle:.1f}°")
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path and 'out' in locals():
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    if exercise_type == "double":
        min_elbow_angle = (np.min(elbow_angles_left) + np.min(elbow_angles_right)) / 2 if elbow_angles_left and elbow_angles_right else 180
        max_elbow_angle = (np.max(elbow_angles_left) + np.max(elbow_angles_right)) / 2 if elbow_angles_left and elbow_angles_right else 0
        avg_shoulder_angle = (np.mean(shoulder_angles_left) + np.mean(shoulder_angles_right)) / 2 if shoulder_angles_left and shoulder_angles_right else 0
    elif active_arm == "left":
        min_elbow_angle = np.min(elbow_angles_left) if elbow_angles_left else 180
        max_elbow_angle = np.max(elbow_angles_left) if elbow_angles_left else 0
        avg_shoulder_angle = np.mean(shoulder_angles_left) if shoulder_angles_left else 0
    else:  # right arm
        min_elbow_angle = np.min(elbow_angles_right) if elbow_angles_right else 180
        max_elbow_angle = np.max(elbow_angles_right) if elbow_angles_right else 0
        avg_shoulder_angle = np.mean(shoulder_angles_right) if shoulder_angles_right else 0
    
    avg_torso_angle = np.mean(torso_angles) if torso_angles else 0
    
    # Calculate wrist alignment (lower is better)
    avg_wrist_alignment = np.mean(wrist_alignments) if wrist_alignments else 45
    wrist_alignment_score = max(0, 100 - (avg_wrist_alignment * 2))  # Convert to 0-100 score
    
    # Calculate head position stability
    head_stability = np.std(head_positions) * 100 if head_positions else 0
    head_stability_score = max(0, 100 - (head_stability * 10))  # Convert to 0-100 score
    
    # Check for key tricep extension form criteria
    full_range_of_motion = min_elbow_angle < 80 and max_elbow_angle > 160
    upper_arm_vertical = abs(avg_shoulder_angle - 90) < 20  # Upper arm should stay close to vertical
    torso_stable = avg_torso_angle < 15  # Torso should stay upright
    wrists_proper = wrist_alignment_score > 70  # Wrists should stay aligned with forearms
    head_stable = head_stability_score > 70  # Head should stay in stable position
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "exercise_type": exercise_type,
            "active_arm": active_arm if exercise_type == "single" else "both",
            "min_elbow_angle": min_elbow_angle,    # Lower value means better range of motion
            "max_elbow_angle": max_elbow_angle,    # Higher value means better extension
            "upper_arm_angle": avg_shoulder_angle, # Should be close to 90° (vertical)
            "torso_angle": avg_torso_angle,        # Should be close to 0° (upright)
            "wrist_alignment": wrist_alignment_score,  # Higher is better (0-100)
            "head_stability": head_stability_score,    # Higher is better (0-100)
            "full_range_of_motion": full_range_of_motion,
            "upper_arm_vertical": upper_arm_vertical,
            "torso_stable": torso_stable,
            "wrists_proper": wrists_proper,
            "head_stable": head_stable,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    # Check range of motion
    if min_elbow_angle > 80:
        feedback["feedback"].append("Try to bend your elbows more at the bottom of the movement. You're not getting a full range of motion.")
    
    if max_elbow_angle < 160:
        feedback["feedback"].append("Extend your arms more fully at the top of the movement to fully engage your triceps.")
    
    # Check upper arm position
    if not upper_arm_vertical:
        if avg_shoulder_angle < 70:
            feedback["feedback"].append("Keep your upper arms more vertical. You're letting your elbows drift too far forward.")
        else:
            feedback["feedback"].append("Keep your upper arms more vertical. You're letting your elbows drift too far backward.")
    
    # Check torso position
    if not torso_stable:
        feedback["feedback"].append("Try to keep your torso more stable. Avoid leaning forward or arching your back during the extension.")
    
    # Check wrist alignment
    if not wrists_proper:
        feedback["feedback"].append("Keep your wrists straight and aligned with your forearms throughout the movement.")
    
    # Check head position
    if not head_stable:
        feedback["feedback"].append("Keep your head in a stable position. Avoid moving it forward or backward as you extend your arms.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your tricep extensions show good range of motion, stable upper arms, and proper extension.")
        
    return feedback

# Example usage - Make sure you have the correct file path
video_file = "clips/TricepExtension1.MOV"  # Update this to the correct path
result = analyze_tricep_extension(video_file, "clips/analyzed_triceps1.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Exercise type: {result['form_analysis']['exercise_type']}-arm tricep extension")
    print(f"Active arm: {result['form_analysis']['active_arm']}")
    print(f"Min elbow angle (lower is better): {result['form_analysis']['min_elbow_angle']:.1f}°")
    print(f"Max elbow angle (higher is better): {result['form_analysis']['max_elbow_angle']:.1f}°")
    print(f"Upper arm angle (should be ~90°): {result['form_analysis']['upper_arm_angle']:.1f}°")
    print(f"Torso angle (lower is better): {result['form_analysis']['torso_angle']:.1f}°")
    print(f"Wrist alignment score: {result['form_analysis']['wrist_alignment']:.1f}/100")
    print(f"Head stability score: {result['form_analysis']['head_stability']:.1f}/100")
    print(f"Full range of motion: {'Yes' if result['form_analysis']['full_range_of_motion'] else 'No'}")
    print(f"Upper arms stayed vertical: {'Yes' if result['form_analysis']['upper_arm_vertical'] else 'No'}")
    print(f"Torso stayed stable: {'Yes' if result['form_analysis']['torso_stable'] else 'No'}")
    print(f"Wrists properly aligned: {'Yes' if result['form_analysis']['wrists_proper'] else 'No'}")
    print(f"Head stayed stable: {'Yes' if result['form_analysis']['head_stable'] else 'No'}")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745948578.852465 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745948578.945771 6360067 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745948578.959545 6360069 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29, Total frames: 373
Detected 33 landmarks
Detected double-arm tricep extension with both arm(s)
Rep #1 detected at frame with average elbow angle 171.4°
Rep #2 detected at frame with average elbow angle 164.5°
Processing frame 100/373
Rep #3 detected at frame with average elbow angle 165.7°
Rep #4 detected at frame with average elbow angle 162.3°
Processing frame 200/373
Rep #5 detected at frame with average elbow angle 165.2°
Processing frame 300/373
Rep #6 detected at frame with average elbow angle 164.0°

=== ANALYSIS RESULTS ===
Counted 6 reps
Exercise type: double-arm tricep extension
Active arm: both
Min elbow angle (lower is better): 17.2°
Max elbow angle (higher is better): 179.7°
Upper arm angle (should be ~90°): 75.1°
Torso angle (lower is better): 81.0°
Wrist alignment score: 15.2/100
Head stability score: 94.2/100
Full range of motion: Yes
Upper arms stayed vertical: Yes
Torso stayed stable: No
Wrists properly aligned: No
Head stayed stab

In [17]:
import mediapipe as mp
import cv2
import numpy as np
import math
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_tricep_extension(video_path, output_video_path=None):
    # Check if file exists
    if not os.path.isfile(video_path):
        print(f"Error: File '{video_path}' does not exist!")
        return {
            "rep_count": 0,
            "error": f"Video file not found: {video_path}"
        }
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return {
            "rep_count": 0,
            "error": f"Could not open video file: {video_path}. Check file format and codecs."
        }
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {total_frames}")
    
    # If we still have 0x0 dimensions, there's a codec issue
    if frame_width == 0 or frame_height == 0:
        print(f"Error: Could not determine video dimensions. Try converting the video to MP4 format.")
        return {
            "rep_count": 0,
            "error": "Could not determine video dimensions. Try converting to MP4 format."
        }
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or try 'avc1'
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track tricep extension state
    rep_count = 0
    extension_stage = None  # "up" (arms extended) or "down" (arms bent)
    good_frames = 0
    
    # Determine if it's two-arm or one-arm extension by analyzing initial frames
    exercise_type = None    # "single" or "double"
    active_arm = None       # "left" or "right" (for single-arm only)
    
    # Lists to store angles and positions for analysis
    elbow_angles_left = []
    elbow_angles_right = []
    shoulder_angles_left = [] # To check if upper arm stays vertical/stationary
    shoulder_angles_right = []
    wrist_alignments = []   # To track wrist alignment with forearms
    torso_angles = []       # To track torso stability
    head_positions = []     # To track head position
    
    frame_count = 0
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get all key points for tricep extension analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Get hip and head points for posture analysis
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                   landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            
            # Calculate midpoints
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, 
                          (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, 
                      (left_hip[1] + right_hip[1])/2]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Calculate elbow angles (shoulder-elbow-wrist)
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            
            elbow_angles_left.append(left_elbow_angle)
            elbow_angles_right.append(right_elbow_angle)
            
            # Calculate shoulder angles to check if upper arm stays vertical
            # Create vertical reference points directly below shoulders
            left_vertical_ref = [left_shoulder[0], left_shoulder[1] + 0.2]
            right_vertical_ref = [right_shoulder[0], right_shoulder[1] + 0.2]
            
            # Calculate angle between shoulder, elbow, and vertical reference
            left_shoulder_angle = calculate_angle(left_vertical_ref, left_shoulder, left_elbow)
            right_shoulder_angle = calculate_angle(right_vertical_ref, right_shoulder, right_elbow)
            
            shoulder_angles_left.append(left_shoulder_angle)
            shoulder_angles_right.append(right_shoulder_angle)
            
            # Calculate torso angle relative to vertical
            # Create a point directly below mid_hip to represent vertical
            vertical_point = [mid_hip[0], mid_hip[1] + 0.2]  # Adding to y moves down in image coords
            torso_angle = calculate_angle(mid_shoulder, mid_hip, vertical_point)
            torso_angle = 180 - torso_angle if torso_angle > 90 else torso_angle
            torso_angles.append(torso_angle)
            
            # Track head position relative to shoulders
            head_to_shoulder_distance = math.sqrt((nose[0] - mid_shoulder[0])**2 + 
                                                 (nose[1] - mid_shoulder[1])**2)
            head_positions.append(head_to_shoulder_distance)
            
            # Determine exercise type based on elbow movement in first 30 frames
            if good_frames == 30 and exercise_type is None:
                left_range = max(elbow_angles_left) - min(elbow_angles_left)
                right_range = max(elbow_angles_right) - min(elbow_angles_right)
                
                # If both elbows have significant movement, it's double-arm
                if left_range > 20 and right_range > 20:
                    exercise_type = "double"
                elif left_range > right_range:
                    exercise_type = "single"
                    active_arm = "left"
                else:
                    exercise_type = "single"
                    active_arm = "right"
                    
                print(f"Detected {exercise_type}-arm tricep extension with {active_arm if exercise_type == 'single' else 'both'} arm(s)")
            
            # Calculate average wrist alignment
            # For tricep extensions, wrists should stay aligned with forearms
            left_wrist_alignment = abs(left_elbow_angle - 180) if left_elbow_angle > 90 else left_elbow_angle
            right_wrist_alignment = abs(right_elbow_angle - 180) if right_elbow_angle > 90 else right_elbow_angle
            
            if exercise_type == "double":
                avg_wrist_alignment = (left_wrist_alignment + right_wrist_alignment) / 2
            elif active_arm == "left":
                avg_wrist_alignment = left_wrist_alignment
            else:
                avg_wrist_alignment = right_wrist_alignment
                
            wrist_alignments.append(avg_wrist_alignment)
            
            # Determine which elbow angle to use for rep counting
            if exercise_type == "double":
                avg_elbow_angle = (left_elbow_angle + right_elbow_angle) / 2
            elif active_arm == "left":
                avg_elbow_angle = left_elbow_angle
            else:
                avg_elbow_angle = right_elbow_angle
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            left_elbow_px = normalized_to_pixel_coordinates(left_elbow[0], left_elbow[1], frame_width, frame_height)
            right_elbow_px = normalized_to_pixel_coordinates(right_elbow[0], right_elbow[1], frame_width, frame_height)
            mid_hip_px = normalized_to_pixel_coordinates(mid_hip[0], mid_hip[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"L: {left_elbow_angle:.1f}°",
                        (left_elbow_px[0] - 50, left_elbow_px[1] + 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"R: {right_elbow_angle:.1f}°",
                        (right_elbow_px[0] - 50, right_elbow_px[1] + 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Torso: {torso_angle:.1f}°",
                        (mid_hip_px[0] - 50, mid_hip_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine extension stage based on elbow angle
            # For tricep extensions, "down" is when elbows are bent
            # "up" is when arms are extended
            
            # Extension is "down" when elbow angle is small (arms bent, weight behind head)
            if avg_elbow_angle < 90 and (extension_stage == "up" or extension_stage is None):
                extension_stage = "down"
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # Extension is "up" when elbow angle is large (arms extended)
            elif avg_elbow_angle > 160 and extension_stage == "down":
                extension_stage = "up"
                # Coming from "down", count a rep
                rep_count += 1
                print(f"Rep #{rep_count} detected at frame with average elbow angle {avg_elbow_angle:.1f}°")
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path and 'out' in locals():
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    if exercise_type == "double":
        min_elbow_angle = (np.min(elbow_angles_left) + np.min(elbow_angles_right)) / 2 if elbow_angles_left and elbow_angles_right else 180
        max_elbow_angle = (np.max(elbow_angles_left) + np.max(elbow_angles_right)) / 2 if elbow_angles_left and elbow_angles_right else 0
        avg_shoulder_angle = (np.mean(shoulder_angles_left) + np.mean(shoulder_angles_right)) / 2 if shoulder_angles_left and shoulder_angles_right else 0
    elif active_arm == "left":
        min_elbow_angle = np.min(elbow_angles_left) if elbow_angles_left else 180
        max_elbow_angle = np.max(elbow_angles_left) if elbow_angles_left else 0
        avg_shoulder_angle = np.mean(shoulder_angles_left) if shoulder_angles_left else 0
    else:  # right arm
        min_elbow_angle = np.min(elbow_angles_right) if elbow_angles_right else 180
        max_elbow_angle = np.max(elbow_angles_right) if elbow_angles_right else 0
        avg_shoulder_angle = np.mean(shoulder_angles_right) if shoulder_angles_right else 0
    
    avg_torso_angle = np.mean(torso_angles) if torso_angles else 0
    
    # Calculate wrist alignment (lower is better)
    avg_wrist_alignment = np.mean(wrist_alignments) if wrist_alignments else 45
    wrist_alignment_score = max(0, 100 - (avg_wrist_alignment * 2))  # Convert to 0-100 score
    
    # Calculate head position stability
    head_stability = np.std(head_positions) * 100 if head_positions else 0
    head_stability_score = max(0, 100 - (head_stability * 10))  # Convert to 0-100 score
    
    # Check for key tricep extension form criteria
    full_range_of_motion = min_elbow_angle < 80 and max_elbow_angle > 160
    upper_arm_vertical = abs(avg_shoulder_angle - 90) < 20  # Upper arm should stay close to vertical
    torso_stable = avg_torso_angle < 15  # Torso should stay upright
    wrists_proper = wrist_alignment_score > 70  # Wrists should stay aligned with forearms
    head_stable = head_stability_score > 70  # Head should stay in stable position
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "exercise_type": exercise_type,
            "active_arm": active_arm if exercise_type == "single" else "both",
            "min_elbow_angle": min_elbow_angle,    # Lower value means better range of motion
            "max_elbow_angle": max_elbow_angle,    # Higher value means better extension
            "upper_arm_angle": avg_shoulder_angle, # Should be close to 90° (vertical)
            "torso_angle": avg_torso_angle,        # Should be close to 0° (upright)
            "wrist_alignment": wrist_alignment_score,  # Higher is better (0-100)
            "head_stability": head_stability_score,    # Higher is better (0-100)
            "full_range_of_motion": full_range_of_motion,
            "upper_arm_vertical": upper_arm_vertical,
            "torso_stable": torso_stable,
            "wrists_proper": wrists_proper,
            "head_stable": head_stable,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    # Check range of motion
    if min_elbow_angle > 80:
        feedback["feedback"].append("Try to bend your elbows more at the bottom of the movement. You're not getting a full range of motion.")
    
    if max_elbow_angle < 160:
        feedback["feedback"].append("Extend your arms more fully at the top of the movement to fully engage your triceps.")
    
    # Check upper arm position
    if not upper_arm_vertical:
        if avg_shoulder_angle < 70:
            feedback["feedback"].append("Keep your upper arms more vertical. You're letting your elbows drift too far forward.")
        else:
            feedback["feedback"].append("Keep your upper arms more vertical. You're letting your elbows drift too far backward.")
    
    # Check torso position
    if not torso_stable:
        feedback["feedback"].append("Try to keep your torso more stable. Avoid leaning forward or arching your back during the extension.")
    
    # Check wrist alignment
    if not wrists_proper:
        feedback["feedback"].append("Keep your wrists straight and aligned with your forearms throughout the movement.")
    
    # Check head position
    if not head_stable:
        feedback["feedback"].append("Keep your head in a stable position. Avoid moving it forward or backward as you extend your arms.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your tricep extensions show good range of motion, stable upper arms, and proper extension.")
        
    return feedback

# Example usage - Make sure you have the correct file path
video_file = "clips/TricepExtension2.MOV"  # Update this to the correct path
result = analyze_tricep_extension(video_file, "clips/analyzed_triceps2.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Exercise type: {result['form_analysis']['exercise_type']}-arm tricep extension")
    print(f"Active arm: {result['form_analysis']['active_arm']}")
    print(f"Min elbow angle (lower is better): {result['form_analysis']['min_elbow_angle']:.1f}°")
    print(f"Max elbow angle (higher is better): {result['form_analysis']['max_elbow_angle']:.1f}°")
    print(f"Upper arm angle (should be ~90°): {result['form_analysis']['upper_arm_angle']:.1f}°")
    print(f"Torso angle (lower is better): {result['form_analysis']['torso_angle']:.1f}°")
    print(f"Wrist alignment score: {result['form_analysis']['wrist_alignment']:.1f}/100")
    print(f"Head stability score: {result['form_analysis']['head_stability']:.1f}/100")
    print(f"Full range of motion: {'Yes' if result['form_analysis']['full_range_of_motion'] else 'No'}")
    print(f"Upper arms stayed vertical: {'Yes' if result['form_analysis']['upper_arm_vertical'] else 'No'}")
    print(f"Torso stayed stable: {'Yes' if result['form_analysis']['torso_stable'] else 'No'}")
    print(f"Wrists properly aligned: {'Yes' if result['form_analysis']['wrists_proper'] else 'No'}")
    print(f"Head stayed stable: {'Yes' if result['form_analysis']['head_stable'] else 'No'}")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745948614.126786 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745948614.218453 6361084 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745948614.229012 6361084 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29, Total frames: 373
Detected 33 landmarks
Rep #1 detected at frame with average elbow angle 160.2°
Detected double-arm tricep extension with both arm(s)
Processing frame 100/373
Processing frame 200/373
Processing frame 300/373

=== ANALYSIS RESULTS ===
Counted 1 reps
Exercise type: double-arm tricep extension
Active arm: both
Min elbow angle (lower is better): 16.2°
Max elbow angle (higher is better): 158.7°
Upper arm angle (should be ~90°): 27.3°
Torso angle (lower is better): 86.1°
Wrist alignment score: 2.3/100
Head stability score: 84.0/100
Full range of motion: No
Upper arms stayed vertical: No
Torso stayed stable: No
Wrists properly aligned: No
Head stayed stable: Yes
Frames analyzed: 373

Feedback:
- Extend your arms more fully at the top of the movement to fully engage your triceps.
- Keep your upper arms more vertical. You're letting your elbows drift too far forward.
- Try to keep your torso more stable. Avoid leaning forward or arching yo

In [18]:
import mediapipe as mp
import cv2
import numpy as np
import math
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_tricep_extension(video_path, output_video_path=None):
    # Check if file exists
    if not os.path.isfile(video_path):
        print(f"Error: File '{video_path}' does not exist!")
        return {
            "rep_count": 0,
            "error": f"Video file not found: {video_path}"
        }
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return {
            "rep_count": 0,
            "error": f"Could not open video file: {video_path}. Check file format and codecs."
        }
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {total_frames}")
    
    # If we still have 0x0 dimensions, there's a codec issue
    if frame_width == 0 or frame_height == 0:
        print(f"Error: Could not determine video dimensions. Try converting the video to MP4 format.")
        return {
            "rep_count": 0,
            "error": "Could not determine video dimensions. Try converting to MP4 format."
        }
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or try 'avc1'
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track tricep extension state
    rep_count = 0
    extension_stage = None  # "up" (arms extended) or "down" (arms bent)
    good_frames = 0
    
    # Determine if it's two-arm or one-arm extension by analyzing initial frames
    exercise_type = None    # "single" or "double"
    active_arm = None       # "left" or "right" (for single-arm only)
    
    # Lists to store angles and positions for analysis
    elbow_angles_left = []
    elbow_angles_right = []
    shoulder_angles_left = [] # To check if upper arm stays vertical/stationary
    shoulder_angles_right = []
    wrist_alignments = []   # To track wrist alignment with forearms
    torso_angles = []       # To track torso stability
    head_positions = []     # To track head position
    
    frame_count = 0
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get all key points for tricep extension analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Get hip and head points for posture analysis
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                   landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            
            # Calculate midpoints
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, 
                          (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, 
                      (left_hip[1] + right_hip[1])/2]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Calculate elbow angles (shoulder-elbow-wrist)
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            
            elbow_angles_left.append(left_elbow_angle)
            elbow_angles_right.append(right_elbow_angle)
            
            # Calculate shoulder angles to check if upper arm stays vertical
            # Create vertical reference points directly below shoulders
            left_vertical_ref = [left_shoulder[0], left_shoulder[1] + 0.2]
            right_vertical_ref = [right_shoulder[0], right_shoulder[1] + 0.2]
            
            # Calculate angle between shoulder, elbow, and vertical reference
            left_shoulder_angle = calculate_angle(left_vertical_ref, left_shoulder, left_elbow)
            right_shoulder_angle = calculate_angle(right_vertical_ref, right_shoulder, right_elbow)
            
            shoulder_angles_left.append(left_shoulder_angle)
            shoulder_angles_right.append(right_shoulder_angle)
            
            # Calculate torso angle relative to vertical
            # Create a point directly below mid_hip to represent vertical
            vertical_point = [mid_hip[0], mid_hip[1] + 0.2]  # Adding to y moves down in image coords
            torso_angle = calculate_angle(mid_shoulder, mid_hip, vertical_point)
            torso_angle = 180 - torso_angle if torso_angle > 90 else torso_angle
            torso_angles.append(torso_angle)
            
            # Track head position relative to shoulders
            head_to_shoulder_distance = math.sqrt((nose[0] - mid_shoulder[0])**2 + 
                                                 (nose[1] - mid_shoulder[1])**2)
            head_positions.append(head_to_shoulder_distance)
            
            # Determine exercise type based on elbow movement in first 30 frames
            if good_frames == 30 and exercise_type is None:
                left_range = max(elbow_angles_left) - min(elbow_angles_left)
                right_range = max(elbow_angles_right) - min(elbow_angles_right)
                
                # If both elbows have significant movement, it's double-arm
                if left_range > 20 and right_range > 20:
                    exercise_type = "double"
                elif left_range > right_range:
                    exercise_type = "single"
                    active_arm = "left"
                else:
                    exercise_type = "single"
                    active_arm = "right"
                    
                print(f"Detected {exercise_type}-arm tricep extension with {active_arm if exercise_type == 'single' else 'both'} arm(s)")
            
            # Calculate average wrist alignment
            # For tricep extensions, wrists should stay aligned with forearms
            left_wrist_alignment = abs(left_elbow_angle - 180) if left_elbow_angle > 90 else left_elbow_angle
            right_wrist_alignment = abs(right_elbow_angle - 180) if right_elbow_angle > 90 else right_elbow_angle
            
            if exercise_type == "double":
                avg_wrist_alignment = (left_wrist_alignment + right_wrist_alignment) / 2
            elif active_arm == "left":
                avg_wrist_alignment = left_wrist_alignment
            else:
                avg_wrist_alignment = right_wrist_alignment
                
            wrist_alignments.append(avg_wrist_alignment)
            
            # Determine which elbow angle to use for rep counting
            if exercise_type == "double":
                avg_elbow_angle = (left_elbow_angle + right_elbow_angle) / 2
            elif active_arm == "left":
                avg_elbow_angle = left_elbow_angle
            else:
                avg_elbow_angle = right_elbow_angle
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize key angles
            left_elbow_px = normalized_to_pixel_coordinates(left_elbow[0], left_elbow[1], frame_width, frame_height)
            right_elbow_px = normalized_to_pixel_coordinates(right_elbow[0], right_elbow[1], frame_width, frame_height)
            mid_hip_px = normalized_to_pixel_coordinates(mid_hip[0], mid_hip[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"L: {left_elbow_angle:.1f}°",
                        (left_elbow_px[0] - 50, left_elbow_px[1] + 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"R: {right_elbow_angle:.1f}°",
                        (right_elbow_px[0] - 50, right_elbow_px[1] + 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(annotated_image, 
                        f"Torso: {torso_angle:.1f}°",
                        (mid_hip_px[0] - 50, mid_hip_px[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine extension stage based on elbow angle
            # For tricep extensions, "down" is when elbows are bent
            # "up" is when arms are extended
            
            # Extension is "down" when elbow angle is small (arms bent, weight behind head)
            if avg_elbow_angle < 90 and (extension_stage == "up" or extension_stage is None):
                extension_stage = "down"
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # Extension is "up" when elbow angle is large (arms extended)
            elif avg_elbow_angle > 160 and extension_stage == "down":
                extension_stage = "up"
                # Coming from "down", count a rep
                rep_count += 1
                print(f"Rep #{rep_count} detected at frame with average elbow angle {avg_elbow_angle:.1f}°")
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path and 'out' in locals():
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    if exercise_type == "double":
        min_elbow_angle = (np.min(elbow_angles_left) + np.min(elbow_angles_right)) / 2 if elbow_angles_left and elbow_angles_right else 180
        max_elbow_angle = (np.max(elbow_angles_left) + np.max(elbow_angles_right)) / 2 if elbow_angles_left and elbow_angles_right else 0
        avg_shoulder_angle = (np.mean(shoulder_angles_left) + np.mean(shoulder_angles_right)) / 2 if shoulder_angles_left and shoulder_angles_right else 0
    elif active_arm == "left":
        min_elbow_angle = np.min(elbow_angles_left) if elbow_angles_left else 180
        max_elbow_angle = np.max(elbow_angles_left) if elbow_angles_left else 0
        avg_shoulder_angle = np.mean(shoulder_angles_left) if shoulder_angles_left else 0
    else:  # right arm
        min_elbow_angle = np.min(elbow_angles_right) if elbow_angles_right else 180
        max_elbow_angle = np.max(elbow_angles_right) if elbow_angles_right else 0
        avg_shoulder_angle = np.mean(shoulder_angles_right) if shoulder_angles_right else 0
    
    avg_torso_angle = np.mean(torso_angles) if torso_angles else 0
    
    # Calculate wrist alignment (lower is better)
    avg_wrist_alignment = np.mean(wrist_alignments) if wrist_alignments else 45
    wrist_alignment_score = max(0, 100 - (avg_wrist_alignment * 2))  # Convert to 0-100 score
    
    # Calculate head position stability
    head_stability = np.std(head_positions) * 100 if head_positions else 0
    head_stability_score = max(0, 100 - (head_stability * 10))  # Convert to 0-100 score
    
    # Check for key tricep extension form criteria
    full_range_of_motion = min_elbow_angle < 80 and max_elbow_angle > 160
    upper_arm_vertical = abs(avg_shoulder_angle - 90) < 20  # Upper arm should stay close to vertical
    torso_stable = avg_torso_angle < 15  # Torso should stay upright
    wrists_proper = wrist_alignment_score > 70  # Wrists should stay aligned with forearms
    head_stable = head_stability_score > 70  # Head should stay in stable position
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "exercise_type": exercise_type,
            "active_arm": active_arm if exercise_type == "single" else "both",
            "min_elbow_angle": min_elbow_angle,    # Lower value means better range of motion
            "max_elbow_angle": max_elbow_angle,    # Higher value means better extension
            "upper_arm_angle": avg_shoulder_angle, # Should be close to 90° (vertical)
            "torso_angle": avg_torso_angle,        # Should be close to 0° (upright)
            "wrist_alignment": wrist_alignment_score,  # Higher is better (0-100)
            "head_stability": head_stability_score,    # Higher is better (0-100)
            "full_range_of_motion": full_range_of_motion,
            "upper_arm_vertical": upper_arm_vertical,
            "torso_stable": torso_stable,
            "wrists_proper": wrists_proper,
            "head_stable": head_stable,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    # Check range of motion
    if min_elbow_angle > 80:
        feedback["feedback"].append("Try to bend your elbows more at the bottom of the movement. You're not getting a full range of motion.")
    
    if max_elbow_angle < 160:
        feedback["feedback"].append("Extend your arms more fully at the top of the movement to fully engage your triceps.")
    
    # Check upper arm position
    if not upper_arm_vertical:
        if avg_shoulder_angle < 70:
            feedback["feedback"].append("Keep your upper arms more vertical. You're letting your elbows drift too far forward.")
        else:
            feedback["feedback"].append("Keep your upper arms more vertical. You're letting your elbows drift too far backward.")
    
    # Check torso position
    if not torso_stable:
        feedback["feedback"].append("Try to keep your torso more stable. Avoid leaning forward or arching your back during the extension.")
    
    # Check wrist alignment
    if not wrists_proper:
        feedback["feedback"].append("Keep your wrists straight and aligned with your forearms throughout the movement.")
    
    # Check head position
    if not head_stable:
        feedback["feedback"].append("Keep your head in a stable position. Avoid moving it forward or backward as you extend your arms.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your tricep extensions show good range of motion, stable upper arms, and proper extension.")
        
    return feedback

# Example usage - Make sure you have the correct file path
video_file = "clips/TricepExtension3.MOV"  # Update this to the correct path
result = analyze_tricep_extension(video_file, "clips/analyzed_triceps3.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('rep_count', 0)} reps")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Exercise type: {result['form_analysis']['exercise_type']}-arm tricep extension")
    print(f"Active arm: {result['form_analysis']['active_arm']}")
    print(f"Min elbow angle (lower is better): {result['form_analysis']['min_elbow_angle']:.1f}°")
    print(f"Max elbow angle (higher is better): {result['form_analysis']['max_elbow_angle']:.1f}°")
    print(f"Upper arm angle (should be ~90°): {result['form_analysis']['upper_arm_angle']:.1f}°")
    print(f"Torso angle (lower is better): {result['form_analysis']['torso_angle']:.1f}°")
    print(f"Wrist alignment score: {result['form_analysis']['wrist_alignment']:.1f}/100")
    print(f"Head stability score: {result['form_analysis']['head_stability']:.1f}/100")
    print(f"Full range of motion: {'Yes' if result['form_analysis']['full_range_of_motion'] else 'No'}")
    print(f"Upper arms stayed vertical: {'Yes' if result['form_analysis']['upper_arm_vertical'] else 'No'}")
    print(f"Torso stayed stable: {'Yes' if result['form_analysis']['torso_stable'] else 'No'}")
    print(f"Wrists properly aligned: {'Yes' if result['form_analysis']['wrists_proper'] else 'No'}")
    print(f"Head stayed stable: {'Yes' if result['form_analysis']['head_stable'] else 'No'}")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1745948637.674162 6301017 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1745948637.791337 6361771 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745948637.803816 6361771 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 30, Total frames: 166
Detected 33 landmarks
Detected double-arm tricep extension with both arm(s)
Processing frame 100/166
Rep #1 detected at frame with average elbow angle 160.5°

=== ANALYSIS RESULTS ===
Counted 1 reps
Exercise type: double-arm tricep extension
Active arm: both
Min elbow angle (lower is better): 15.6°
Max elbow angle (higher is better): 168.0°
Upper arm angle (should be ~90°): 51.9°
Torso angle (lower is better): 79.1°
Wrist alignment score: 9.1/100
Head stability score: 61.9/100
Full range of motion: Yes
Upper arms stayed vertical: No
Torso stayed stable: No
Wrists properly aligned: No
Head stayed stable: No
Frames analyzed: 165

Feedback:
- Keep your upper arms more vertical. You're letting your elbows drift too far forward.
- Try to keep your torso more stable. Avoid leaning forward or arching your back during the extension.
- Keep your wrists straight and aligned with your forearms throughout the movement.
- Keep your head in a s